# Data

## season_game_level_data

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import datetime, time
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
from statsmodels.formula.api import ols
from statsmodels.iolib.summary2 import summary_col
from pylab import hist, show
import scipy
import zipfile


pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 200)

**data frames used in this notebook:**
- da = pbp
- dg = season_games_data
- dm = play_by_play
- dp = player_rankings
- dw = team_roster_player_rank
- dv = season_team_roster_ranking
- dx = season_game_team_roster
- dz = season_team
- dy = season_game_roster

**for analysis:**
- ds = season_level
- dl = season_game_level
- dt = season_game_team_level

In [2]:
pwd

'/Users/stefanostselios/Desktop/nhl_roster_design-master'

### import play by play data set

In [3]:
da = pd.read_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/pbp_merged.csv')
#da = pd.read_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/pbp_merged.csv')
da = da.drop('Unnamed: 0', axis=1)
da = da.rename(columns={'TeamCode': 'EventTeamCode'})


keep regular season games

In [4]:
da = da[da['GameNumber'] <= 21230]

drop irrelevant data

In [5]:
da = da[da['EventType']!='STOP']
da = da[da['EventType']!='EISTR']
da = da[da['EventType']!='EIEND']

- display goals for each game and drop duplicates.

In [6]:
dh = da[da['EventTeamCode']==da['HTeamCode']]
dh = dh.rename(columns={'EventTeamCode': 'HTeam'})
dh['goal'] = dh.apply(lambda x: 1 if (x['EventType'] == 'GOAL') else 0, axis=1)
dh['HGF'] = dh.groupby(['Season', 'GameNumber', 'HTeam'])['goal'].transform('sum')
dh = dh[['Season', 'GameNumber', 'HGF']]
dh = dh.drop_duplicates(['Season', 'GameNumber'])

In [7]:
dv = da[da['EventTeamCode']==da['VTeamCode']]
dv = dv.rename(columns={'EventTeamCode': 'VTeam'})
dv['goal'] = dv.apply(lambda x: 1 if (x['EventType'] == 'GOAL') else 0, axis=1)
dv['VGF'] = dv.groupby(['Season', 'GameNumber', 'VTeam'])['goal'].transform('sum')
dv = dv[['Season', 'GameNumber', 'VGF']]
dv = dv.drop_duplicates(['Season', 'GameNumber'])

Merge into season-game data

In [8]:
dg = da[['Season', 'GameNumber', 'VTeamCode', 'HTeamCode']]
dg = dg.drop_duplicates(['Season', 'GameNumber'])

In [9]:
dg = pd.merge(dg, dh, on=['Season', 'GameNumber'], how='left')
dg = pd.merge(dg, dv, on=['Season', 'GameNumber'], how='left')

- find the goal differential per game with respect to home team.

In [10]:
dg['GD'] = dg['HGF'] - dg['VGF']
dg['WinTeam'] = dg.apply(lambda x: x['HTeamCode'] if x['GD'] > 0 else x['VTeamCode'], axis=1)
dg['LossTeam'] = dg.apply(lambda x: x['HTeamCode'] if x['WinTeam'] != x['HTeamCode'] else x['VTeamCode'], axis=1)

In [11]:
dg.head()

,Season,GameNumber,VTeamCode,HTeamCode,HGF,VGF,GD,WinTeam,LossTeam
0,2010,20001,MTL,TOR,3,2,1,TOR,MTL
1,2010,20002,PHI,PIT,2,3,-1,PHI,PIT
2,2010,20003,CAR,MIN,3,4,-1,CAR,MIN
3,2010,20004,CHI,COL,4,3,1,COL,CHI
4,2010,20005,CGY,EDM,4,0,4,EDM,CGY


- display goals againest per team.

In [12]:
dg['VGA'] = dg['HGF']
dg['HGA'] = dg['VGF']

In [13]:
dg = dg[['Season', 'GameNumber', 'VTeamCode', 'HTeamCode', 'VGF', 'HGF', 'GD', 'VGA', 'HGA', 'WinTeam', 'LossTeam']]
dg.head()

,Season,GameNumber,VTeamCode,HTeamCode,VGF,HGF,GD,VGA,HGA,WinTeam,LossTeam
0,2010,20001,MTL,TOR,2,3,1,3,2,TOR,MTL
1,2010,20002,PHI,PIT,3,2,-1,2,3,PHI,PIT
2,2010,20003,CAR,MIN,4,3,-1,3,4,CAR,MIN
3,2010,20004,CHI,COL,3,4,1,4,3,COL,CHI
4,2010,20005,CGY,EDM,0,4,4,4,0,EDM,CGY


In [23]:
dg.to_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/data/season_game_data.csv', index='False', sep=',')
#dg.to_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/data/season_game_data.csv', index='False', sep=',')

## season_level_data

In [24]:
dm = da.copy()

events that happened in regulation time only

In [25]:
dm = dm[dm['Period'] <= 3]
dm = dm[dm['Period'] >= 1]

- **reshape the data set from wide to long.**

In [26]:
dm = dm.sort_values(['Season', 'GameNumber', 'Period', 'EventNumber'], ascending=[True, True, True, True])

In [27]:
a = [col for col in dm.columns if 'VPlayer' in col]
b = [col for col in dm.columns if 'HPlayer' in col]
c = [col for col in dm.columns if 'VPosition' in col]
d = [col for col in dm.columns if 'HPosition' in col]
dm = pd.lreshape(dm, {'VPlayer' : a, 'HPlayer' : b, 'VPosition' : c, 'HPosition': d})

In [28]:
dm.shape

(1796745, 24)

In [29]:
dm = dm.rename(columns={'PlayerNumber': 'EventPlayerNumber', 'TeamCode': 'EventTeamCode', 'PlayerName': 'EventPlayerName' })
dm = dm[['Season', 'GameNumber', 'GameDate', 'Period', 'AdvantageType', 'Zone', 'EventNumber', 'EventType', 'EventDetail', 'EventTeamCode', 'EventPlayerNumber', 'EventPlayerName', 'EventTimeFromZero', 'EventTimeFromTwenty', 'VTeamCode', 'VPlayer', 'VPosition', 'HTeamCode', 'HPlayer', 'HPosition', 'ShotType', 'ShotResult', 'Length', 'PenaltyType']]
dm = dm.sort_values(['Season', 'GameNumber', 'Period', 'EventNumber'], ascending=[True, True, True, True])

- fill in advantage type with even strength 'EV' and event player number with 'TEAM'

In [30]:
dm['AdvantageType'] = dm['AdvantageType'].fillna('EV')
dm['EventPlayerNumber'] = dm['EventPlayerNumber'].fillna('TEAM')

- save new datast as play by play

In [31]:
dm.to_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/data/play_by_play.csv', index='False', sep=',')
#dm.to_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/data/play_by_play.csv', index='False', sep=',')

#### create new data set and keep variables: 
- (a) game number.
- (b) visitor team information.
- (c) home team information.

In [32]:
df = dm.copy()
df = df[['Season', 'GameNumber', 'VTeamCode', 'VPlayer', 'VPosition', 'HTeamCode', 'HPlayer', 'HPosition']]
df = df.sort_values(['Season', 'GameNumber'], ascending=[True, True])
df.head()

,Season,GameNumber,VTeamCode,VPlayer,VPosition,HTeamCode,HPlayer,HPosition
0,2010,20001,MTL,11.0,C,TOR,37.0,C
310113,2010,20001,MTL,21.0,R,TOR,9.0,R
620126,2010,20001,MTL,57.0,L,TOR,11.0,L
930061,2010,20001,MTL,26.0,D,TOR,3.0,D
1239931,2010,20001,MTL,75.0,D,TOR,22.0,D


- merge season_game_data (dg) on new dataset

In [33]:
df = pd.merge(df, dg, on=['Season', 'GameNumber', 'VTeamCode', 'HTeamCode'], how='left')
df.head()    

,Season,GameNumber,VTeamCode,VPlayer,VPosition,HTeamCode,HPlayer,HPosition,VGF,HGF,GD,VGA,HGA,WinTeam,LossTeam
0,2010,20001,MTL,11.0,C,TOR,37.0,C,2,3,1,3,2,TOR,MTL
1,2010,20001,MTL,21.0,R,TOR,9.0,R,2,3,1,3,2,TOR,MTL
2,2010,20001,MTL,57.0,L,TOR,11.0,L,2,3,1,3,2,TOR,MTL
3,2010,20001,MTL,26.0,D,TOR,3.0,D,2,3,1,3,2,TOR,MTL
4,2010,20001,MTL,75.0,D,TOR,22.0,D,2,3,1,3,2,TOR,MTL


- reshape the data to have home and visitor team observatons under the same coloumns. 

In [34]:
a = [col for col in df.columns if 'Player' in col]
b = [col for col in df.columns if 'Position' in col]
c = [col for col in df.columns if 'TeamCode' in col]
d = [col for col in df.columns if 'GF' in col]
e = [col for col in df.columns if 'GA' in col]
df = pd.lreshape(df, {'PlayerNumber' : a, 'PlayerPosition' : b, 'TeamCode' : c, 'GF' : d, 'GA' : e })
df = df[['Season', 'GameNumber', 'TeamCode', 'PlayerNumber', 'GF', 'GA', 'GD', 'WinTeam', 'LossTeam']]
df.head()

,Season,GameNumber,TeamCode,PlayerNumber,GF,GA,GD,WinTeam,LossTeam
0,2010,20001,MTL,11.0,2,3,1,TOR,MTL
1,2010,20001,MTL,21.0,2,3,1,TOR,MTL
2,2010,20001,MTL,57.0,2,3,1,TOR,MTL
3,2010,20001,MTL,26.0,2,3,1,TOR,MTL
4,2010,20001,MTL,75.0,2,3,1,TOR,MTL


### import player position and rankings

In [36]:
dp = pd.read_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/clusters/stats_per_player_nhl_positions.csv')
#dp = pd.read_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/clusters/stats_per_player_nhl_positions.csv')
dp = dp.drop('Unnamed: 0', axis=1)
dp = dp.rename(columns={'Position': 'PlayerPosition'})

In [37]:
dp = dp[['Season', 'TeamCode', 'PlayerNumber', 'PlayerName', 'PlayerPosition']]
dp.head()

,Season,TeamCode,PlayerNumber,PlayerName,PlayerPosition
0,2010,MTL,11.0,GOMEZ,C
1,2010,TOR,37.0,BRENT,C
2,2010,MTL,14.0,PLEKANEC,C
3,2010,MTL,76.0,SUBBAN,D
4,2010,TOR,35.0,GIGUERE,G


In [38]:
dp.shape

(1058, 5)

In [39]:
dr = pd.read_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/clusters/player_rank_nhl_positions.csv')
#dr = pd.read_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/clusters/player_rank_nhl_positions.csv')
dr = dr.drop('Unnamed: 0', axis=1)
dr = dr.rename(columns={'Position': 'PlayerPosition'})

In [40]:
dr = dr[['Season', 'TeamCode', 'PlayerNumber', 'PlayerName', 'PlayerPosition', 'Rank']]
dr = dr.sort_values(['TeamCode'], ascending=[True])
dr.head()

,Season,TeamCode,PlayerNumber,PlayerName,PlayerPosition,Rank
222,2010,ANA,63.0,BONINO,C,4
793,2010,ANA,34.0,VOROS,W,4
645,2010,ANA,19.0,LUPUL,W,3
153,2010,ANA,11.0,KOIVU,C,3
351,2010,ANA,23.0,BEAUCHEMIN,D,2


- merge player position and player rankings and drop goaltenders

In [41]:
ds = pd.merge(dp, dr, on=['Season', 'TeamCode', 'PlayerNumber', 'PlayerName', 'PlayerPosition'], how='left')
ds['Rank'] = ds.groupby(['Season', 'TeamCode', 'PlayerName'])['Rank'].apply(lambda x: x.ffill().bfill())
ds.shape

(1058, 6)

In [42]:
ds.isnull().sum()

Season              0
TeamCode            0
PlayerNumber        0
PlayerName          0
PlayerPosition      0
Rank              256
dtype: int64

skaters that have played less than 9 games were not included in the clusters and therefore not ranked. Since the failed to make the roster on a regular basis, forwards are assigned to the 4th line and defensemen to the bottom (3rd) pairing.

In [43]:
ds['Rank'] = ds.apply(lambda x: 1 if ((x['PlayerPosition'] == 'G') & (pd.isnull(x.Rank))) else 3 if ((x['PlayerPosition'] == 'D') & (pd.isnull(x.Rank))) else 4 if ((x['PlayerPosition'] == 'W') & (pd.isnull(x.Rank))) else 4 if ((x['PlayerPosition'] == 'C') & (pd.isnull(x.Rank))) else x['Rank'], axis=1)
ds = ds.sort_values(['TeamCode'], ascending=[True])
ds.shape

(1058, 6)

In [44]:
ds1 = ds[(ds['PlayerPosition'] == 'D') & (ds['Rank'] == 4)]
ds1.head()

,Season,TeamCode,PlayerNumber,PlayerName,PlayerPosition,Rank
996,2010,OTT,51.0,SMITH,D,4.0


- Derek Smith, a defenseman for the Ottawa Senators has a ranking of 4, which is incorrect since we have 3 defensive pairings. For that reason, he is assigned a rank of 3 which represents the bottom defensive pairing

In [45]:
ds['Rank'] = ds.apply(lambda x: 3 if ((x['PlayerPosition'] == 'D') & (x['TeamCode'] == 'OTT') & (x['PlayerName'] == 'SMITH') & (x['PlayerNumber'] == 51.0)) else x['Rank'], axis=1)

In [46]:
ds1 = ds[(ds['PlayerPosition'] == 'D') & (ds['Rank'] == 4)]
ds1.head()

,Season,TeamCode,PlayerNumber,PlayerName,PlayerPosition,Rank


- **display each player by team per game. Drop duplicates.**

In [47]:
dw = pd.merge(df, ds, on=['Season', 'TeamCode', 'PlayerNumber'], how='left')
dw = dw[['Season', 'GameNumber', 'TeamCode', 'PlayerNumber', 'PlayerName', 'PlayerPosition', 'Rank', 'GF', 'GA', 'GD', 'WinTeam', 'LossTeam']]
dw.head()

,Season,GameNumber,TeamCode,PlayerNumber,PlayerName,PlayerPosition,Rank,GF,GA,GD,WinTeam,LossTeam
0,2010,20001,MTL,11.0,GOMEZ,C,3.0,2,3,1,TOR,MTL
1,2010,20001,MTL,21.0,GIONTA,W,2.0,2,3,1,TOR,MTL
2,2010,20001,MTL,57.0,POULIOT,W,2.0,2,3,1,TOR,MTL
3,2010,20001,MTL,26.0,GORGES,D,3.0,2,3,1,TOR,MTL
4,2010,20001,MTL,75.0,GILL,D,3.0,2,3,1,TOR,MTL


- drop observations that have no player name, position nor ranking. Exclude goaltenders from the roster.

In [48]:
dw = dw[pd.notnull(dw['PlayerPosition'])]
dw.isnull().sum()

Season            0
GameNumber        0
TeamCode          0
PlayerNumber      0
PlayerName        0
PlayerPosition    0
Rank              0
GF                0
GA                0
GD                0
WinTeam           0
LossTeam          0
dtype: int64

In [49]:
dw.shape

(3688734, 12)

In [50]:
dw = dw[dw['PlayerPosition'] != 'G']
dw.shape

(3130215, 12)

- create column that displays the position and roster count by team per game. To simplify matters, we categorize skaters into forwards and defensemen.

In [51]:
dw = dw.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerNumber'])
dw['RosterCount'] = dw.groupby(['Season', 'GameNumber', 'TeamCode'])['PlayerNumber'].transform('count')
dw['PositionCount'] = dw.groupby(['Season', 'GameNumber', 'TeamCode', 'PlayerPosition'])['PlayerNumber'].transform('count')
dw.head()

,Season,GameNumber,TeamCode,PlayerNumber,PlayerName,PlayerPosition,Rank,GF,GA,GD,WinTeam,LossTeam,RosterCount,PositionCount
0,2010,20001,MTL,11.0,GOMEZ,C,3.0,2,3,1,TOR,MTL,18.0,7.0
1,2010,20001,MTL,21.0,GIONTA,W,2.0,2,3,1,TOR,MTL,18.0,5.0
2,2010,20001,MTL,57.0,POULIOT,W,2.0,2,3,1,TOR,MTL,18.0,5.0
3,2010,20001,MTL,26.0,GORGES,D,3.0,2,3,1,TOR,MTL,18.0,6.0
4,2010,20001,MTL,75.0,GILL,D,3.0,2,3,1,TOR,MTL,18.0,6.0


- count the amount of forwards and defensemen by team per game.

In [52]:
dw['CCount'] = dw.apply(lambda x: x['PositionCount'] if x['PlayerPosition']=='C' else np.NaN, 1)
dw['WCount'] = dw.apply(lambda x: x['PositionCount'] if x['PlayerPosition']=='W' else np.NaN, 1)
dw['DCount'] = dw.apply(lambda x: x['PositionCount'] if x['PlayerPosition']=='D' else np.NaN, 1)
dw['CCount'] = dw.groupby(['Season','GameNumber', 'TeamCode'])['CCount'].apply(lambda x: x.ffill().bfill())
dw['WCount'] = dw.groupby(['Season','GameNumber', 'TeamCode'])['WCount'].apply(lambda x: x.ffill().bfill())
dw['DCount'] = dw.groupby(['Season','GameNumber', 'TeamCode'])['DCount'].apply(lambda x: x.ffill().bfill())
dw.head()

,Season,GameNumber,TeamCode,PlayerNumber,PlayerName,PlayerPosition,Rank,GF,GA,GD,WinTeam,LossTeam,RosterCount,PositionCount,CCount,WCount,DCount
0,2010,20001,MTL,11.0,GOMEZ,C,3.0,2,3,1,TOR,MTL,18.0,7.0,7.0,5.0,6.0
1,2010,20001,MTL,21.0,GIONTA,W,2.0,2,3,1,TOR,MTL,18.0,5.0,7.0,5.0,6.0
2,2010,20001,MTL,57.0,POULIOT,W,2.0,2,3,1,TOR,MTL,18.0,5.0,7.0,5.0,6.0
3,2010,20001,MTL,26.0,GORGES,D,3.0,2,3,1,TOR,MTL,18.0,6.0,7.0,5.0,6.0
4,2010,20001,MTL,75.0,GILL,D,3.0,2,3,1,TOR,MTL,18.0,6.0,7.0,5.0,6.0


In [219]:
dw.to_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/data/full_season_team_game_player_rank_nhl_positions.csv', index='False', sep=',')
#dw.to_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/data/full_season_team_game_player_rank_nhl_positions.csv', index='False', sep=',')

In [220]:
dw1 = dw.drop_duplicates(['Season', 'GameNumber', 'TeamCode'])
dw1 = dw1.sort_values(['Season', 'GameNumber'], ascending=[True, True])

In [221]:
dw.head()

,Season,GameNumber,TeamCode,PlayerNumber,PlayerName,PlayerPosition,Rank,GF,GA,GD,WinTeam,LossTeam,RosterCount,PositionCount,CCount,WCount,DCount
0,2010,20001,MTL,11.0,GOMEZ,C,3.0,2,3,1,TOR,MTL,18.0,7.0,7.0,5.0,6.0
1,2010,20001,MTL,21.0,GIONTA,W,2.0,2,3,1,TOR,MTL,18.0,5.0,7.0,5.0,6.0
2,2010,20001,MTL,57.0,POULIOT,W,2.0,2,3,1,TOR,MTL,18.0,5.0,7.0,5.0,6.0
3,2010,20001,MTL,26.0,GORGES,D,3.0,2,3,1,TOR,MTL,18.0,6.0,7.0,5.0,6.0
4,2010,20001,MTL,75.0,GILL,D,3.0,2,3,1,TOR,MTL,18.0,6.0,7.0,5.0,6.0


In [222]:
dw1.shape

(2460, 17)

In [223]:
dw1.head()

,Season,GameNumber,TeamCode,PlayerNumber,PlayerName,PlayerPosition,Rank,GF,GA,GD,WinTeam,LossTeam,RosterCount,PositionCount,CCount,WCount,DCount
0,2010,20001,MTL,11.0,GOMEZ,C,3.0,2,3,1,TOR,MTL,18.0,7.0,7.0,5.0,6.0
1847320,2010,20001,TOR,37.0,BRENT,C,4.0,3,2,1,TOR,MTL,18.0,5.0,5.0,7.0,6.0
1611,2010,20002,PHI,17.0,CARTER,C,1.0,3,2,-1,PHI,PIT,18.0,5.0,5.0,7.0,6.0
1848965,2010,20002,PIT,71.0,MALKIN,C,1.0,2,3,-1,PHI,PIT,18.0,8.0,8.0,4.0,6.0
3224,2010,20003,CAR,53.0,SKINNER,C,1.0,4,3,-1,CAR,MIN,18.0,6.0,6.0,6.0,6.0


In [224]:
dw1.to_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/data/team_roster_player_rank_nhl_positions.csv', index='False', sep=',')
#dw1.to_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/data/team_roster_player_rank_nhl_positions.csv', index='False', sep=',')

In [225]:
dw1.shape

(2460, 17)

### full regular season stats

In [226]:
dw2 = dw.copy()
dw2 = dw2.groupby(['Season', 'GameNumber', 'TeamCode', 'PlayerPosition', 'GF', 'GA', 'RosterCount', 'PositionCount', 'CCount', 'WCount', 'DCount', 'WinTeam', 'LossTeam'], as_index=False)['Rank'].mean()
dw2.shape

(7380, 14)

- create columns for team win and team loss. 

In [227]:
dw2['TeamWin'] =  dw2.apply(lambda x: 1 if x['TeamCode']==x['WinTeam'] else 0, 1)
dw2['TeamLos'] =  dw2.apply(lambda x: 1 if x['TeamCode']==x['LossTeam'] else 0, 1)

- display games played, games won, games loss, goals for and goals against by team for the season.

In [228]:
dw2['GP'] = dw2.groupby(['Season', 'PlayerPosition', 'TeamCode'])['GameNumber'].transform('count')
dw2['GW'] = dw2.groupby(['Season', 'PlayerPosition', 'TeamCode'])['TeamWin'].transform('sum')
dw2['GL'] = dw2.groupby(['Season', 'PlayerPosition', 'TeamCode'])['TeamLos'].transform('sum')
dw2['GF'] = dw2.groupby(['Season', 'PlayerPosition', 'TeamCode'])['GF'].transform('sum')
dw2['GA'] = dw2.groupby(['Season', 'PlayerPosition', 'TeamCode'])['GA'].transform('sum')
dw2.head()

,Season,GameNumber,TeamCode,PlayerPosition,GF,GA,RosterCount,PositionCount,CCount,WCount,DCount,WinTeam,LossTeam,Rank,TeamWin,TeamLos,GP,GW,GL
0,2010,20001,MTL,C,220,213,18.0,7.0,7.0,5.0,6.0,TOR,MTL,2.714286,0,1,82,44,38
1,2010,20001,MTL,D,220,213,18.0,6.0,7.0,5.0,6.0,TOR,MTL,2.166667,0,1,82,44,38
2,2010,20001,MTL,W,220,213,18.0,5.0,7.0,5.0,6.0,TOR,MTL,2.600000,0,1,82,44,38
3,2010,20001,TOR,C,225,259,18.0,5.0,5.0,7.0,6.0,TOR,MTL,2.400000,1,0,82,37,45
4,2010,20001,TOR,D,225,259,18.0,6.0,5.0,7.0,6.0,TOR,MTL,2.000000,1,0,82,37,45


- create columns with the mean ranking for forward and defenseman by team per game.

In [229]:
dw2['RankC'] = dw2.apply(lambda x: x['Rank'] if x['PlayerPosition']=='C' else np.NaN, 1)
dw2['RankW'] = dw2.apply(lambda x: x['Rank'] if x['PlayerPosition']=='W' else np.NaN, 1)
dw2['RankD'] = dw2.apply(lambda x: x['Rank'] if x['PlayerPosition']=='D' else np.NaN, 1)
dw2['RankC'] = dw2.groupby(['Season','GameNumber', 'TeamCode'])['RankC'].apply(lambda x: x.ffill().bfill())
dw2['RankW'] = dw2.groupby(['Season','GameNumber', 'TeamCode'])['RankW'].apply(lambda x: x.ffill().bfill())
dw2['RankD'] = dw2.groupby(['Season','GameNumber', 'TeamCode'])['RankD'].apply(lambda x: x.ffill().bfill())
dw2.head()

,Season,GameNumber,TeamCode,PlayerPosition,GF,GA,RosterCount,PositionCount,CCount,WCount,DCount,WinTeam,LossTeam,Rank,TeamWin,TeamLos,GP,GW,GL,RankC,RankW,RankD
0,2010,20001,MTL,C,220,213,18.0,7.0,7.0,5.0,6.0,TOR,MTL,2.714286,0,1,82,44,38,2.714286,2.600000,2.166667
1,2010,20001,MTL,D,220,213,18.0,6.0,7.0,5.0,6.0,TOR,MTL,2.166667,0,1,82,44,38,2.714286,2.600000,2.166667
2,2010,20001,MTL,W,220,213,18.0,5.0,7.0,5.0,6.0,TOR,MTL,2.600000,0,1,82,44,38,2.714286,2.600000,2.166667
3,2010,20001,TOR,C,225,259,18.0,5.0,5.0,7.0,6.0,TOR,MTL,2.400000,1,0,82,37,45,2.400000,2.571429,2.000000
4,2010,20001,TOR,D,225,259,18.0,6.0,5.0,7.0,6.0,TOR,MTL,2.000000,1,0,82,37,45,2.400000,2.571429,2.000000


- compute the mean per position by team for the season.

In [230]:
dw2['MeanC']= dw2.groupby(['Season', 'TeamCode'])['RankC'].transform('mean')
dw2['MeanW']= dw2.groupby(['Season', 'TeamCode'])['RankW'].transform('mean')
dw2['MeanD']= dw2.groupby(['Season', 'TeamCode'])['RankD'].transform('mean')
dw2.head()

,Season,GameNumber,TeamCode,PlayerPosition,GF,GA,RosterCount,PositionCount,CCount,WCount,DCount,WinTeam,LossTeam,Rank,TeamWin,TeamLos,GP,GW,GL,RankC,RankW,RankD,MeanC,MeanW,MeanD
0,2010,20001,MTL,C,220,213,18.0,7.0,7.0,5.0,6.0,TOR,MTL,2.714286,0,1,82,44,38,2.714286,2.600000,2.166667,2.577584,2.539663,2.226481
1,2010,20001,MTL,D,220,213,18.0,6.0,7.0,5.0,6.0,TOR,MTL,2.166667,0,1,82,44,38,2.714286,2.600000,2.166667,2.577584,2.539663,2.226481
2,2010,20001,MTL,W,220,213,18.0,5.0,7.0,5.0,6.0,TOR,MTL,2.600000,0,1,82,44,38,2.714286,2.600000,2.166667,2.577584,2.539663,2.226481
3,2010,20001,TOR,C,225,259,18.0,5.0,5.0,7.0,6.0,TOR,MTL,2.400000,1,0,82,37,45,2.400000,2.571429,2.000000,2.169919,2.704646,2.224158
4,2010,20001,TOR,D,225,259,18.0,6.0,5.0,7.0,6.0,TOR,MTL,2.000000,1,0,82,37,45,2.400000,2.571429,2.000000,2.169919,2.704646,2.224158


- display the quantity of wins and losses per team for the whole season

In [231]:
dw2['L'] = dw2.apply(lambda x: x['GL'] if x['TeamCode']== x['LossTeam'] else (x['GP'] - x['GW']), 1)
dw2['W'] = dw2.apply(lambda x: x['GW'] if x['TeamCode']== x['WinTeam'] else (x['GP'] - x['GL']), 1)
dw2.head()

,Season,GameNumber,TeamCode,PlayerPosition,GF,GA,RosterCount,PositionCount,CCount,WCount,DCount,WinTeam,LossTeam,Rank,TeamWin,TeamLos,GP,GW,GL,RankC,RankW,RankD,MeanC,MeanW,MeanD,L,W
0,2010,20001,MTL,C,220,213,18.0,7.0,7.0,5.0,6.0,TOR,MTL,2.714286,0,1,82,44,38,2.714286,2.600000,2.166667,2.577584,2.539663,2.226481,38,44
1,2010,20001,MTL,D,220,213,18.0,6.0,7.0,5.0,6.0,TOR,MTL,2.166667,0,1,82,44,38,2.714286,2.600000,2.166667,2.577584,2.539663,2.226481,38,44
2,2010,20001,MTL,W,220,213,18.0,5.0,7.0,5.0,6.0,TOR,MTL,2.600000,0,1,82,44,38,2.714286,2.600000,2.166667,2.577584,2.539663,2.226481,38,44
3,2010,20001,TOR,C,225,259,18.0,5.0,5.0,7.0,6.0,TOR,MTL,2.400000,1,0,82,37,45,2.400000,2.571429,2.000000,2.169919,2.704646,2.224158,45,37
4,2010,20001,TOR,D,225,259,18.0,6.0,5.0,7.0,6.0,TOR,MTL,2.000000,1,0,82,37,45,2.400000,2.571429,2.000000,2.169919,2.704646,2.224158,45,37


- compute win and loss percent by team. Drop duplicate observations.

In [232]:
dw2 = dw2[['Season', 'TeamCode', 'GP', 'L', 'W', 'GF', 'GA', 'MeanC', 'MeanW', 'MeanD']]
dw2 = dw2.drop_duplicates(['Season', 'TeamCode'])
dw2['WinPc'] = dw2['W']/ dw2['GP']
dw2['LossPc'] = dw2['L']/ dw2['GP']

dw2 = dw2[['Season', 'TeamCode', 'GP','W', 'L', 'GF', 'GA', 'WinPc', 'LossPc', 'MeanC', 'MeanW', 'MeanD']]
dw2.head()

,Season,TeamCode,GP,W,L,GF,GA,WinPc,LossPc,MeanC,MeanW,MeanD
0,2010,MTL,82,44,38,220,213,0.536585,0.463415,2.577584,2.539663,2.226481
3,2010,TOR,82,37,45,225,259,0.451220,0.548780,2.169919,2.704646,2.224158
6,2010,PHI,82,47,35,266,233,0.573171,0.426829,1.996951,2.192073,1.735772
9,2010,PIT,82,49,33,244,203,0.597561,0.402439,2.320093,2.650697,2.022358
12,2010,CAR,82,40,42,239,242,0.487805,0.512195,2.012805,2.224158,1.963415


- rank teams based on win percent, mean centres, wingers and defensemen. 

In [233]:
dw2['RankWin'] = dw2.groupby(['Season'])['WinPc'].rank(ascending=False)
dw2['RankC'] = dw2.groupby(['Season'])['MeanC'].rank(ascending=True)
dw2['RankW'] = dw2.groupby(['Season'])['MeanW'].rank(ascending=True)
dw2['RankD'] = dw2.groupby(['Season'])['MeanD'].rank(ascending=True)
dw2 = dw2.sort_values(['Season', 'RankWin', 'RankC', 'RankW', 'RankD'], ascending=[True, True, True, True, True])
dw2.head(30)

,Season,TeamCode,GP,W,L,GF,GA,WinPc,LossPc,MeanC,MeanW,MeanD,RankWin,RankC,RankW,RankD
135,2010,VAN,82,54,28,268,190,0.658537,0.341463,2.296748,2.136929,2.228804,1.0,9.0,5.0,14.0
9,2010,PIT,82,49,33,244,203,0.597561,0.402439,2.320093,2.650697,2.022358,2.0,10.0,24.0,8.0
33,2010,SJ,82,48,34,253,216,0.585366,0.414634,1.775465,2.519541,2.382259,3.5,1.0,18.0,17.0
63,2010,WSH,82,48,34,230,203,0.585366,0.414634,2.700958,2.200450,2.521196,3.5,21.0,11.0,26.0
51,2010,DET,82,47,35,263,241,0.573171,0.426829,1.859059,2.170267,1.813298,6.0,2.0,7.0,2.0
6,2010,PHI,82,47,35,266,233,0.573171,0.426829,1.996951,2.192073,1.735772,6.0,5.0,9.0,1.0
48,2010,ANA,82,47,35,241,237,0.573171,0.426829,2.996022,2.293670,2.104820,6.0,27.0,13.0,10.0
105,2010,TB,82,46,36,252,246,0.560976,0.439024,1.895528,2.037456,2.506678,9.0,4.0,3.0,24.0
132,2010,LA,82,46,36,227,207,0.560976,0.439024,2.493089,2.532424,2.241580,9.0,14.0,19.0,16.0
66,2010,BOS,82,46,36,250,200,0.560976,0.439024,2.630139,2.002787,1.989837,9.0,19.0,2.0,6.0


In [234]:
dw2.to_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/data/full_season_team_roster_ranking_nhl_positions.csv', index='False', sep=',')
#dw2.to_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/data/full_season_team_roster_ranking_nhl_positions.csv', index='False', sep=',')

### keep games that have 12 forwards and 6 defensemen per team!!

In [235]:
dwfd = dw.copy()
dwfd.shape

(44262, 17)

In [236]:
dwfd = dwfd.groupby(['Season', 'GameNumber']).filter(lambda x: ((x['RosterCount'] == 18) & (x['DCount'] == 6)).all())
dwfd.shape

(30636, 17)

In [237]:
dwfd.to_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/data/12f_6d_game_team_player_ranking_nhl_positions.csv', index='False', sep=',')
#dwfd.to_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/data/12f_6d_game_team_player_ranking_nhl_positions.csv', index='False', sep=',')

In [238]:
dw3 = dwfd.copy()
dw3 = dw3.groupby(['Season', 'GameNumber', 'TeamCode', 'PlayerPosition', 'GF', 'GA', 'RosterCount', 'PositionCount', 'CCount', 'WCount', 'DCount', 'WinTeam', 'LossTeam'], as_index=False)['Rank'].mean()
dw3.head()

,Season,GameNumber,TeamCode,PlayerPosition,GF,GA,RosterCount,PositionCount,CCount,WCount,DCount,WinTeam,LossTeam,Rank
0,2010,20001,MTL,C,2,3,18.0,7.0,7.0,5.0,6.0,TOR,MTL,2.714286
1,2010,20001,MTL,D,2,3,18.0,6.0,7.0,5.0,6.0,TOR,MTL,2.166667
2,2010,20001,MTL,W,2,3,18.0,5.0,7.0,5.0,6.0,TOR,MTL,2.600000
3,2010,20001,TOR,C,3,2,18.0,5.0,5.0,7.0,6.0,TOR,MTL,2.400000
4,2010,20001,TOR,D,3,2,18.0,6.0,5.0,7.0,6.0,TOR,MTL,2.000000


In [239]:
dw3.shape

(5106, 14)

- create columns for team win and team loss. 

In [240]:
dw3['TeamWin'] =  dw3.apply(lambda x: 1 if x['TeamCode']==x['WinTeam'] else 0, 1)
dw3['TeamLos'] =  dw3.apply(lambda x: 1 if x['TeamCode']==x['LossTeam'] else 0, 1)

- display games played, games won, games loss, goals for and goals against by team for the season.

In [241]:
dw3['GP'] = dw3.groupby(['Season', 'PlayerPosition', 'TeamCode'])['GameNumber'].transform('count')
dw3['GW'] = dw3.groupby(['Season', 'PlayerPosition', 'TeamCode'])['TeamWin'].transform('sum')
dw3['GL'] = dw3.groupby(['Season', 'PlayerPosition', 'TeamCode'])['TeamLos'].transform('sum')
dw3['GF'] = dw3.groupby(['Season', 'PlayerPosition', 'TeamCode'])['GF'].transform('sum')
dw3['GA'] = dw3.groupby(['Season', 'PlayerPosition', 'TeamCode'])['GA'].transform('sum')

- create columns with the mean ranking for forward and defenseman by team per game.

In [242]:
dw3['RankC'] = dw3.apply(lambda x: x['Rank'] if x['PlayerPosition']=='C' else np.NaN, 1)
dw3['RankW'] = dw3.apply(lambda x: x['Rank'] if x['PlayerPosition']=='W' else np.NaN, 1)
dw3['RankD'] = dw3.apply(lambda x: x['Rank'] if x['PlayerPosition']=='D' else np.NaN, 1)
dw3['RankC'] = dw3.groupby(['Season','GameNumber', 'TeamCode'])['RankC'].apply(lambda x: x.ffill().bfill())
dw3['RankW'] = dw3.groupby(['Season','GameNumber', 'TeamCode'])['RankW'].apply(lambda x: x.ffill().bfill())
dw3['RankD'] = dw3.groupby(['Season','GameNumber', 'TeamCode'])['RankD'].apply(lambda x: x.ffill().bfill())

- compute the mean per position by team for the season.

In [243]:
dw3['MeanC']= dw3.groupby(['Season', 'TeamCode'])['RankC'].transform('mean')
dw3['MeanW']= dw3.groupby(['Season', 'TeamCode'])['RankW'].transform('mean')
dw3['MeanD']= dw3.groupby(['Season', 'TeamCode'])['RankD'].transform('mean')

- display the quantity of wins and losses per team ( roster of 12 forwards and 6 defensemen)

In [244]:
dw3['L'] = dw3.apply(lambda x: x['GL'] if x['TeamCode']== x['LossTeam'] else (x['GP'] - x['GW']), 1)
dw3['W'] = dw3.apply(lambda x: x['GW'] if x['TeamCode']== x['WinTeam'] else (x['GP'] - x['GL']), 1)

- compute win and loss percent by team. Drop duplicate observations.

In [245]:
dw3 = dw3[['Season', 'TeamCode', 'GP', 'L', 'W', 'GF', 'GA', 'MeanC', 'MeanW', 'MeanD']]
dw3 = dw3.drop_duplicates(['Season', 'TeamCode'])
dw3['WinPc'] = dw3['W']/ dw3['GP']
dw3['LossPc'] = dw3['L']/ dw3['GP']

dw3 = dw3[['Season', 'TeamCode', 'GP','W', 'L', 'GF', 'GA', 'WinPc', 'LossPc', 'MeanC', 'MeanW', 'MeanD']]

- rank teams based on win percent, mean forwards and mean defensemen. 

In [246]:
dw3['RankWin'] = dw3.groupby(['Season'])['WinPc'].rank(ascending=False)
dw3['RankC'] = dw3.groupby(['Season'])['MeanC'].rank(ascending=True)
dw3['RankW'] = dw3.groupby(['Season'])['MeanW'].rank(ascending=True)
dw3['RankD'] = dw3.groupby(['Season'])['MeanD'].rank(ascending=True)
dw3 = dw3.sort_values(['Season', 'RankWin', 'RankC', 'RankW', 'RankD'], ascending=[True, True, True, True, True])
dw3.head(30)

,Season,TeamCode,GP,W,L,GF,GA,WinPc,LossPc,MeanC,MeanW,MeanD,RankWin,RankC,RankW,RankD
267,2010,VAN,59,41,18,201,138,0.694915,0.305085,2.331921,2.146388,2.223164,1.0,10.0,5.0,14.0
45,2010,DET,60,36,24,196,176,0.600000,0.400000,1.884762,2.209841,1.808333,2.0,3.0,11.0,2.0
18,2010,CHI,47,28,19,157,127,0.595745,0.404255,2.643617,1.940012,2.148936,3.0,19.0,1.0,11.0
42,2010,ANA,59,35,24,165,163,0.593220,0.406780,3.040234,2.283757,2.093220,4.0,26.0,13.0,10.0
252,2010,BOS,64,37,27,196,156,0.578125,0.421875,2.617708,1.992411,1.960937,5.0,18.0,2.0,5.0
123,2010,WSH,52,30,22,141,122,0.576923,0.423077,2.723993,2.198855,2.519231,6.0,23.0,9.0,24.0
9,2010,PIT,69,39,30,193,173,0.565217,0.434783,2.331832,2.647550,2.021739,7.0,9.0,24.0,8.0
57,2010,NYR,66,37,29,206,163,0.560606,0.439394,2.357359,2.157756,2.421717,8.5,12.0,6.0,19.0
87,2010,NSH,66,37,29,183,158,0.560606,0.439394,2.717172,2.326840,2.047980,8.5,22.0,15.0,9.0
6,2010,PHI,68,38,30,221,197,0.558824,0.441176,1.996324,2.187500,1.742647,10.0,5.0,8.0,1.0


In [247]:
dw3.to_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/data/season_roster_ranking_nhl_positions.csv', index='False', sep=',')
#dw3.to_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/data/season_roster_ranking_nhl_positions.csv', index='False', sep=',')

### keep games that have 4 C, 8 W  and 6 D per team!!!!

In [248]:
du = dw.groupby(['Season', 'GameNumber']).filter(lambda x: ((x['CCount'] == 4) & (x['WCount'] == 8) & (x['DCount'] == 6)).all())

In [249]:
du.shape

(1584, 17)

In [250]:
du.to_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/data/4c_8w_4d_game_team_player_ranking_nhl_positions.csv', index='False', sep=',')
#du.to_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/data/4c_8w_4d_game_team_player_ranking_nhl_positions.csv', index='False', sep=',')

- create a new dataset using team roster player rank

In [160]:
dv = du.copy()

In [161]:
dv = dv.groupby(['Season', 'GameNumber', 'TeamCode', 'PlayerPosition', 'GF', 'GA', 'RosterCount', 'PositionCount', 'CCount', 'WCount', 'DCount', 'WinTeam', 'LossTeam'], as_index=False)['Rank'].mean()
dv.head(10)

,Season,GameNumber,TeamCode,PlayerPosition,GF,GA,RosterCount,PositionCount,CCount,WCount,DCount,WinTeam,LossTeam,Rank
0,2010,20025,CGY,C,3,1,18.0,4.0,4.0,8.0,6.0,CGY,LA,2.250000
1,2010,20025,CGY,D,3,1,18.0,6.0,4.0,8.0,6.0,CGY,LA,2.333333
2,2010,20025,CGY,W,3,1,18.0,8.0,4.0,8.0,6.0,CGY,LA,2.375000
3,2010,20025,LA,C,1,3,18.0,4.0,4.0,8.0,6.0,CGY,LA,2.750000
4,2010,20025,LA,D,1,3,18.0,6.0,4.0,8.0,6.0,CGY,LA,2.333333
5,2010,20025,LA,W,1,3,18.0,8.0,4.0,8.0,6.0,CGY,LA,2.625000
6,2010,20037,NYI,C,1,2,18.0,4.0,4.0,8.0,6.0,WSH,NYI,3.250000
7,2010,20037,NYI,D,1,2,18.0,6.0,4.0,8.0,6.0,WSH,NYI,2.500000
8,2010,20037,NYI,W,1,2,18.0,8.0,4.0,8.0,6.0,WSH,NYI,2.625000
9,2010,20037,WSH,C,2,1,18.0,4.0,4.0,8.0,6.0,WSH,NYI,2.750000


In [162]:
dv.shape

(264, 14)

- create columns for team win and team loss. 

In [163]:
dv['TeamWin'] =  dv.apply(lambda x: 1 if x['TeamCode']==x['WinTeam'] else 0, 1)
dv['TeamLos'] =  dv.apply(lambda x: 1 if x['TeamCode']==x['LossTeam'] else 0, 1)
dv.head()

,Season,GameNumber,TeamCode,PlayerPosition,GF,GA,RosterCount,PositionCount,CCount,WCount,DCount,WinTeam,LossTeam,Rank,TeamWin,TeamLos
0,2010,20025,CGY,C,3,1,18.0,4.0,4.0,8.0,6.0,CGY,LA,2.250000,1,0
1,2010,20025,CGY,D,3,1,18.0,6.0,4.0,8.0,6.0,CGY,LA,2.333333,1,0
2,2010,20025,CGY,W,3,1,18.0,8.0,4.0,8.0,6.0,CGY,LA,2.375000,1,0
3,2010,20025,LA,C,1,3,18.0,4.0,4.0,8.0,6.0,CGY,LA,2.750000,0,1
4,2010,20025,LA,D,1,3,18.0,6.0,4.0,8.0,6.0,CGY,LA,2.333333,0,1


- display games played, games won, games loss, goals for and goals against by team for the season.

In [164]:
dv['GP'] = dv.groupby(['Season', 'PlayerPosition', 'TeamCode'])['GameNumber'].transform('count')
dv['GW'] = dv.groupby(['Season', 'PlayerPosition', 'TeamCode'])['TeamWin'].transform('sum')
dv['GL'] = dv.groupby(['Season', 'PlayerPosition', 'TeamCode'])['TeamLos'].transform('sum')
dv['GF'] = dv.groupby(['Season', 'PlayerPosition', 'TeamCode'])['GF'].transform('sum')
dv['GA'] = dv.groupby(['Season', 'PlayerPosition', 'TeamCode'])['GA'].transform('sum')
dv.head()

,Season,GameNumber,TeamCode,PlayerPosition,GF,GA,RosterCount,PositionCount,CCount,WCount,DCount,WinTeam,LossTeam,Rank,TeamWin,TeamLos,GP,GW,GL
0,2010,20025,CGY,C,22,25,18.0,4.0,4.0,8.0,6.0,CGY,LA,2.250000,1,0,7,2,5
1,2010,20025,CGY,D,22,25,18.0,6.0,4.0,8.0,6.0,CGY,LA,2.333333,1,0,7,2,5
2,2010,20025,CGY,W,22,25,18.0,8.0,4.0,8.0,6.0,CGY,LA,2.375000,1,0,7,2,5
3,2010,20025,LA,C,13,25,18.0,4.0,4.0,8.0,6.0,CGY,LA,2.750000,0,1,9,2,7
4,2010,20025,LA,D,13,25,18.0,6.0,4.0,8.0,6.0,CGY,LA,2.333333,0,1,9,2,7


- create columns with the mean ranking for forward and defenseman by team per game.

In [165]:
dv['RankC'] = dv.apply(lambda x: x['Rank'] if x['PlayerPosition']=='C' else np.NaN, 1)
dv['RankW'] = dv.apply(lambda x: x['Rank'] if x['PlayerPosition']=='W' else np.NaN, 1)
dv['RankD'] = dv.apply(lambda x: x['Rank'] if x['PlayerPosition']=='D' else np.NaN, 1)
dv['RankC'] = dv.groupby(['Season','GameNumber', 'TeamCode'])['RankC'].apply(lambda x: x.ffill().bfill())
dv['RankW'] = dv.groupby(['Season','GameNumber', 'TeamCode'])['RankW'].apply(lambda x: x.ffill().bfill())
dv['RankD'] = dv.groupby(['Season','GameNumber', 'TeamCode'])['RankD'].apply(lambda x: x.ffill().bfill())
dv.head()

,Season,GameNumber,TeamCode,PlayerPosition,GF,GA,RosterCount,PositionCount,CCount,WCount,DCount,WinTeam,LossTeam,Rank,TeamWin,TeamLos,GP,GW,GL,RankC,RankW,RankD
0,2010,20025,CGY,C,22,25,18.0,4.0,4.0,8.0,6.0,CGY,LA,2.250000,1,0,7,2,5,2.25,2.375,2.333333
1,2010,20025,CGY,D,22,25,18.0,6.0,4.0,8.0,6.0,CGY,LA,2.333333,1,0,7,2,5,2.25,2.375,2.333333
2,2010,20025,CGY,W,22,25,18.0,8.0,4.0,8.0,6.0,CGY,LA,2.375000,1,0,7,2,5,2.25,2.375,2.333333
3,2010,20025,LA,C,13,25,18.0,4.0,4.0,8.0,6.0,CGY,LA,2.750000,0,1,9,2,7,2.75,2.625,2.333333
4,2010,20025,LA,D,13,25,18.0,6.0,4.0,8.0,6.0,CGY,LA,2.333333,0,1,9,2,7,2.75,2.625,2.333333


- compute the mean per position by team for the season.

In [166]:
dv['MeanC']= dv.groupby(['Season', 'TeamCode'])['RankC'].transform('mean')
dv['MeanW']= dv.groupby(['Season', 'TeamCode'])['RankW'].transform('mean')
dv['MeanD']= dv.groupby(['Season', 'TeamCode'])['RankD'].transform('mean')
dv.head()

,Season,GameNumber,TeamCode,PlayerPosition,GF,GA,RosterCount,PositionCount,CCount,WCount,DCount,WinTeam,LossTeam,Rank,TeamWin,TeamLos,GP,GW,GL,RankC,RankW,RankD,MeanC,MeanW,MeanD
0,2010,20025,CGY,C,22,25,18.0,4.0,4.0,8.0,6.0,CGY,LA,2.250000,1,0,7,2,5,2.25,2.375,2.333333,1.821429,2.267857,1.904762
1,2010,20025,CGY,D,22,25,18.0,6.0,4.0,8.0,6.0,CGY,LA,2.333333,1,0,7,2,5,2.25,2.375,2.333333,1.821429,2.267857,1.904762
2,2010,20025,CGY,W,22,25,18.0,8.0,4.0,8.0,6.0,CGY,LA,2.375000,1,0,7,2,5,2.25,2.375,2.333333,1.821429,2.267857,1.904762
3,2010,20025,LA,C,13,25,18.0,4.0,4.0,8.0,6.0,CGY,LA,2.750000,0,1,9,2,7,2.75,2.625,2.333333,2.583333,2.597222,2.240741
4,2010,20025,LA,D,13,25,18.0,6.0,4.0,8.0,6.0,CGY,LA,2.333333,0,1,9,2,7,2.75,2.625,2.333333,2.583333,2.597222,2.240741


- display the quantity of wins and losses per team ( roster of 12 forwards and 6 defensemen)

In [167]:
dv['L'] = dv.apply(lambda x: x['GL'] if x['TeamCode']== x['LossTeam'] else (x['GP'] - x['GW']), 1)
dv['W'] = dv.apply(lambda x: x['GW'] if x['TeamCode']== x['WinTeam'] else (x['GP'] - x['GL']), 1)
dv.head()

,Season,GameNumber,TeamCode,PlayerPosition,GF,GA,RosterCount,PositionCount,CCount,WCount,DCount,WinTeam,LossTeam,Rank,TeamWin,TeamLos,GP,GW,GL,RankC,RankW,RankD,MeanC,MeanW,MeanD,L,W
0,2010,20025,CGY,C,22,25,18.0,4.0,4.0,8.0,6.0,CGY,LA,2.250000,1,0,7,2,5,2.25,2.375,2.333333,1.821429,2.267857,1.904762,5,2
1,2010,20025,CGY,D,22,25,18.0,6.0,4.0,8.0,6.0,CGY,LA,2.333333,1,0,7,2,5,2.25,2.375,2.333333,1.821429,2.267857,1.904762,5,2
2,2010,20025,CGY,W,22,25,18.0,8.0,4.0,8.0,6.0,CGY,LA,2.375000,1,0,7,2,5,2.25,2.375,2.333333,1.821429,2.267857,1.904762,5,2
3,2010,20025,LA,C,13,25,18.0,4.0,4.0,8.0,6.0,CGY,LA,2.750000,0,1,9,2,7,2.75,2.625,2.333333,2.583333,2.597222,2.240741,7,2
4,2010,20025,LA,D,13,25,18.0,6.0,4.0,8.0,6.0,CGY,LA,2.333333,0,1,9,2,7,2.75,2.625,2.333333,2.583333,2.597222,2.240741,7,2


- compute win and loss percent by team. Drop duplicate observations.

In [168]:
dv = dv[['Season', 'TeamCode', 'GP', 'L', 'W', 'GF', 'GA', 'MeanC', 'MeanW', 'MeanD']]
dv = dv.drop_duplicates(['Season', 'TeamCode'])
dv['WinPc'] = dv['W']/ dv['GP']
dv['LossPc'] = dv['L']/ dv['GP']

dv = dv[['Season', 'TeamCode', 'GP','W', 'L', 'GF', 'GA', 'WinPc', 'LossPc', 'MeanC', 'MeanW', 'MeanD']]
dv.head()

,Season,TeamCode,GP,W,L,GF,GA,WinPc,LossPc,MeanC,MeanW,MeanD
0,2010,CGY,7,2,5,22,25,0.285714,0.714286,1.821429,2.267857,1.904762
3,2010,LA,9,2,7,13,25,0.222222,0.777778,2.583333,2.597222,2.240741
6,2010,NYI,5,0,5,8,20,0.000000,1.000000,2.950000,2.575000,2.500000
9,2010,WSH,1,1,0,2,1,1.000000,0.000000,2.750000,2.125000,2.500000
12,2010,MIN,2,1,1,10,7,0.500000,0.500000,2.500000,2.625000,2.500000


- rank teams based on win percent, mean forwards and mean defensemen. 

In [169]:
dv['RankWin'] = dv.groupby(['Season'])['WinPc'].rank(ascending=False)
dv['RankC'] = dv.groupby(['Season'])['MeanC'].rank(ascending=True)
dv['RankW'] = dv.groupby(['Season'])['MeanW'].rank(ascending=True)
dv['RankD'] = dv.groupby(['Season'])['MeanD'].rank(ascending=True)
dv = dv.sort_values(['Season', 'RankWin', 'RankC', 'RankW', 'RankD'], ascending=[True, True, True, True, True])
dv.head(30)

,Season,TeamCode,GP,W,L,GF,GA,WinPc,LossPc,MeanC,MeanW,MeanD,RankWin,RankC,RankW,RankD
78,2010,ANA,6,6,0,18,9,1.000000,0.000000,2.625000,2.479167,2.027778,2.0,10.0,9.0,7.5
9,2010,WSH,1,1,0,2,1,1.000000,0.000000,2.750000,2.125000,2.500000,2.0,11.0,3.0,13.5
66,2010,EDM,2,2,0,6,4,1.000000,0.000000,3.125000,2.687500,2.750000,2.0,18.0,14.0,19.0
15,2010,VAN,9,8,1,34,14,0.888889,0.111111,2.416667,2.152778,2.203704,4.0,6.5,4.0,10.0
72,2010,CAR,9,6,3,30,26,0.666667,0.333333,1.777778,2.361111,1.962963,6.5,1.0,7.0,3.0
120,2010,OTT,6,4,2,23,14,0.666667,0.333333,2.416667,3.104167,2.666667,6.5,6.5,18.0,17.5
114,2010,CHI,6,4,2,21,11,0.666667,0.333333,2.916667,1.750000,2.027778,6.5,12.5,1.0,7.5
99,2010,NJ,3,2,1,8,6,0.666667,0.333333,3.166667,2.625000,2.500000,6.5,19.0,12.5,13.5
12,2010,MIN,2,1,1,10,7,0.500000,0.500000,2.500000,2.625000,2.500000,9.0,8.0,12.5,13.5
48,2010,STL,6,2,4,12,18,0.333333,0.666667,2.125000,2.708333,2.166667,11.0,5.0,15.0,9.0


In [170]:
dv.shape

(19, 16)

In [171]:
dv.to_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/data/season_team_roster_ranking_nhl_positions.csv', index='False', sep=',')
#dv.to_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/data/season_team_roster_ranking_nhl_positions.csv', index='False', sep=',')


## season_game_team_roster_data

- use the team roster player rank dataset (dw) to display the roster quality by team per game

In [172]:
dx1 = du.copy()

In [173]:
dx2 = dv.copy()
dx2 = dx2[['Season', 'TeamCode', 'MeanC', 'MeanW', 'MeanD']]

In [174]:
dx = pd.merge(dx1, dx2, on=['Season', 'TeamCode'], how='left')
dx.head()

,Season,GameNumber,TeamCode,PlayerNumber,PlayerName,PlayerPosition,Rank,GF,GA,GD,WinTeam,LossTeam,RosterCount,PositionCount,CCount,WCount,DCount,MeanC,MeanW,MeanD
0,2010,20025,LA,11.0,KOPITAR,C,1.0,1,3,2,CGY,LA,18.0,4.0,4.0,8.0,6.0,2.583333,2.597222,2.240741
1,2010,20025,LA,23.0,BROWN,W,1.0,1,3,2,CGY,LA,18.0,8.0,4.0,8.0,6.0,2.583333,2.597222,2.240741
2,2010,20025,LA,94.0,SMYTH,W,2.0,1,3,2,CGY,LA,18.0,8.0,4.0,8.0,6.0,2.583333,2.597222,2.240741
3,2010,20025,LA,8.0,DOUGHTY,D,1.0,1,3,2,CGY,LA,18.0,6.0,4.0,8.0,6.0,2.583333,2.597222,2.240741
4,2010,20025,LA,33.0,MITCHELL,D,3.0,1,3,2,CGY,LA,18.0,6.0,4.0,8.0,6.0,2.583333,2.597222,2.240741


In [175]:
dx = dx[['Season', 'GameNumber', 'TeamCode', 'RosterCount', 'PlayerNumber', 'PlayerPosition', 'Rank', 'MeanC', 'MeanW', 'MeanD']]
dx['RankC'] = dx.apply(lambda x: x['Rank'] if x['PlayerPosition']=='C' else np.NaN, 1)
dx['RankW'] = dx.apply(lambda x: x['Rank'] if x['PlayerPosition']=='W' else np.NaN, 1)
dx['RankD'] = dx.apply(lambda x: x['Rank'] if x['PlayerPosition']=='D' else np.NaN, 1)
dx['RankC'] = dx.groupby(['Season','GameNumber', 'TeamCode'])['RankC'].apply(lambda x: x.ffill().bfill())
dx['RankW'] = dx.groupby(['Season','GameNumber', 'TeamCode'])['RankW'].apply(lambda x: x.ffill().bfill())
dx['RankD'] = dx.groupby(['Season','GameNumber', 'TeamCode'])['RankD'].apply(lambda x: x.ffill().bfill())
dx['GMeanC']= dx.groupby(['Season', 'TeamCode'])['RankC'].transform('mean')
dx['GMeanW']= dx.groupby(['Season', 'GameNumber', 'TeamCode'])['RankW'].transform('mean')
dx['GMeanD']= dx.groupby(['Season', 'GameNumber', 'TeamCode'])['RankD'].transform('mean')
dx = dx[['Season', 'GameNumber', 'TeamCode', 'RosterCount', 'PlayerNumber', 'PlayerPosition', 'Rank', 'MeanC', 'MeanW', 'MeanD', 'GMeanC', 'GMeanW', 'GMeanD']]
dx = dx.rename(columns={'PlayerPosition': 'Position'})
dx.head()

,Season,GameNumber,TeamCode,RosterCount,PlayerNumber,Position,Rank,MeanC,MeanW,MeanD,GMeanC,GMeanW,GMeanD
0,2010,20025,LA,18.0,11.0,C,1.0,2.583333,2.597222,2.240741,2.62963,2.722222,2.444444
1,2010,20025,LA,18.0,23.0,W,1.0,2.583333,2.597222,2.240741,2.62963,2.722222,2.444444
2,2010,20025,LA,18.0,94.0,W,2.0,2.583333,2.597222,2.240741,2.62963,2.722222,2.444444
3,2010,20025,LA,18.0,8.0,D,1.0,2.583333,2.597222,2.240741,2.62963,2.722222,2.444444
4,2010,20025,LA,18.0,33.0,D,3.0,2.583333,2.597222,2.240741,2.62963,2.722222,2.444444


- group by season, game number, team and player to count the occurance of each player per game and sum up the observations of players. There should be 18 players per team and 36 per game for the dataset to be correct.

In [176]:
dx['playercount'] = dx.groupby(['Season', 'GameNumber', 'TeamCode', 'PlayerNumber',])['PlayerNumber'].transform('count')

In [177]:
dx['rosterposition'] = dx.groupby(['Season', 'GameNumber', 'TeamCode', 'Position', 'Rank'])['playercount'].transform('sum')

#### pivot table

- the next step is to group players by gamenumber, teamcode, position and rank, to display the quality of players each team has per position. **Pivot table** by player position and rank using roster position values. Game number and team are the indexes. We want to join the levels to generate columns by roster position and rank. 

In [178]:
dx = pd.pivot_table(dx, index=['Season', 'GameNumber', 'TeamCode', 'RosterCount', 'MeanC', 'MeanW', 'MeanD', 'GMeanC', 'GMeanW', 'GMeanD'], columns=['Position', 'Rank'], values=['rosterposition'])
dx = dx.reset_index()
dx.columns = ['_'.join(str(s).strip() for s in col if s) for col in dx.columns]
dx.reset_index()
dx = dx.fillna(0)
dx = dx.rename(columns={'rosterposition_C_1.0': 'C1', 'rosterposition_C_2.0': 'C2', 'rosterposition_C_3.0': 'C3', 'rosterposition_C_4.0': 'C4', 'rosterposition_W_1.0': 'W1', 'rosterposition_W_2.0': 'W2', 'rosterposition_W_3.0': 'W3', 'rosterposition_W_4.0': 'W4', 'rosterposition_D_1.0': 'D1', 'rosterposition_D_2.0': 'D2', 'rosterposition_D_3.0': 'D3' })
dx.head(10)

,Season,GameNumber,TeamCode,RosterCount,MeanC,MeanW,MeanD,GMeanC,GMeanW,GMeanD,C1,C2,C3,C4,D1,D2,D3,W1,W2,W3,W4
0,2010,20025,CGY,18.0,1.821429,2.267857,1.904762,1.777778,1.944444,2.277778,1.0,2.0,0.0,1.0,1.0,2.0,3.0,2.0,3.0,1.0,2.0
1,2010,20025,LA,18.0,2.583333,2.597222,2.240741,2.629630,2.722222,2.444444,1.0,0.0,2.0,1.0,2.0,0.0,4.0,2.0,2.0,1.0,3.0
2,2010,20037,NYI,18.0,2.950000,2.575000,2.500000,2.888889,2.555556,2.500000,0.0,0.0,3.0,1.0,0.0,3.0,3.0,3.0,1.0,0.0,4.0
3,2010,20037,WSH,18.0,2.750000,2.125000,2.500000,2.000000,2.333333,2.500000,1.0,1.0,0.0,2.0,1.0,1.0,4.0,2.0,4.0,1.0,1.0
4,2010,20076,MIN,18.0,2.500000,2.625000,2.500000,2.555556,2.833333,2.500000,1.0,0.0,3.0,0.0,1.0,1.0,4.0,1.0,3.0,2.0,2.0
5,2010,20076,VAN,18.0,2.416667,2.152778,2.203704,2.222222,2.166667,1.944444,2.0,0.0,0.0,2.0,2.0,1.0,3.0,2.0,3.0,1.0,2.0
6,2010,20116,LA,18.0,2.583333,2.597222,2.240741,2.629630,2.388889,2.444444,1.0,0.0,3.0,0.0,1.0,0.0,5.0,2.0,2.0,1.0,3.0
7,2010,20116,MIN,18.0,2.500000,2.625000,2.500000,2.555556,2.777778,2.666667,1.0,0.0,3.0,0.0,1.0,1.0,4.0,1.0,3.0,2.0,2.0
8,2010,20363,CGY,18.0,1.821429,2.267857,1.904762,1.777778,2.055556,1.777778,1.0,3.0,0.0,0.0,2.0,3.0,1.0,2.0,3.0,2.0,1.0
9,2010,20363,VAN,18.0,2.416667,2.152778,2.203704,2.222222,1.777778,1.833333,2.0,0.0,0.0,2.0,2.0,2.0,2.0,3.0,3.0,1.0,1.0


In [179]:
dx.shape

(88, 21)

In [180]:
dx.to_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/data/season_game_team_roster_nhl_positions.csv', index='False', sep=',')
#dx.to_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/data/season_game_team_roster_nhl_positions.csv', index='False', sep=',')

- create a dataset that will display the mean of forwards and defencemen by season per team

- drop duplicates by season and team

In [181]:
dz = dx.copy()
dz = dz.drop_duplicates(['Season', 'TeamCode'])
dz.head()

,Season,GameNumber,TeamCode,RosterCount,MeanC,MeanW,MeanD,GMeanC,GMeanW,GMeanD,C1,C2,C3,C4,D1,D2,D3,W1,W2,W3,W4
0,2010,20025,CGY,18.0,1.821429,2.267857,1.904762,1.777778,1.944444,2.277778,1.0,2.0,0.0,1.0,1.0,2.0,3.0,2.0,3.0,1.0,2.0
1,2010,20025,LA,18.0,2.583333,2.597222,2.240741,2.629630,2.722222,2.444444,1.0,0.0,2.0,1.0,2.0,0.0,4.0,2.0,2.0,1.0,3.0
2,2010,20037,NYI,18.0,2.950000,2.575000,2.500000,2.888889,2.555556,2.500000,0.0,0.0,3.0,1.0,0.0,3.0,3.0,3.0,1.0,0.0,4.0
3,2010,20037,WSH,18.0,2.750000,2.125000,2.500000,2.000000,2.333333,2.500000,1.0,1.0,0.0,2.0,1.0,1.0,4.0,2.0,4.0,1.0,1.0
4,2010,20076,MIN,18.0,2.500000,2.625000,2.500000,2.555556,2.833333,2.500000,1.0,0.0,3.0,0.0,1.0,1.0,4.0,1.0,3.0,2.0,2.0


In [182]:
dz.shape

(19, 21)

In [183]:
#dz.to_csv('season_team.csv', index='False', sep=',')

In [184]:
dz.to_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/data/season_team_nhl_positions.csv', index='False', sep=',')
#dz.to_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/data/season_team_nhl_positions.csv', index='False', sep=',')

## game level data

- create an index variable to deterime if a team is considered visitor or home for a given game. The column will be named "A". The 1st observation per game is the visitor team and will be assigned a value of 1. The 2nd and final observation per game, is the home team, so we fill in NaN with a value of 2 (home team)

In [185]:
dy = dx.copy()

In [186]:
dy.loc[dy.groupby('GameNumber',as_index=False).head(1).index,'A'] = 1
dy = dy.fillna(2)

In [187]:
dy.head()

,Season,GameNumber,TeamCode,RosterCount,MeanC,MeanW,MeanD,GMeanC,GMeanW,GMeanD,C1,C2,C3,C4,D1,D2,D3,W1,W2,W3,W4,A
0,2010,20025,CGY,18.0,1.821429,2.267857,1.904762,1.777778,1.944444,2.277778,1.0,2.0,0.0,1.0,1.0,2.0,3.0,2.0,3.0,1.0,2.0,1.0
1,2010,20025,LA,18.0,2.583333,2.597222,2.240741,2.629630,2.722222,2.444444,1.0,0.0,2.0,1.0,2.0,0.0,4.0,2.0,2.0,1.0,3.0,2.0
2,2010,20037,NYI,18.0,2.950000,2.575000,2.500000,2.888889,2.555556,2.500000,0.0,0.0,3.0,1.0,0.0,3.0,3.0,3.0,1.0,0.0,4.0,1.0
3,2010,20037,WSH,18.0,2.750000,2.125000,2.500000,2.000000,2.333333,2.500000,1.0,1.0,0.0,2.0,1.0,1.0,4.0,2.0,4.0,1.0,1.0,2.0
4,2010,20076,MIN,18.0,2.500000,2.625000,2.500000,2.555556,2.833333,2.500000,1.0,0.0,3.0,0.0,1.0,1.0,4.0,1.0,3.0,2.0,2.0,1.0


- **pivot table using game number as index by whether a team is visitor (1) or home (2)**. The table will display the quality of each player per position and team. The next step is to join columns by team and player quality value. We will have for each team 10 columns ( 5 positions x 2 type of player quality). We will rename the columns as following: VC1 shows the amount of elite centers for the visitor team, HC1 displays the amount of elite centers for the home team etc. We rename the columns and sort them based on team, position and quality. 

In [188]:
dy = pd.pivot_table(dy, index=['Season', 'GameNumber'], columns=['A'], values=['C1', 'C2', 'C3', 'C4', 'W1', 'W2', 'W3', 'W4', 'D1', 'D2', 'D3'])
dy = dy.reset_index()
dy.columns = ['_'.join(str(s).strip() for s in col if s) for col in dy.columns]
dy = dy.reset_index()
dy = dy.rename(columns={'C1_1.0': 'VC1', 'C2_1.0': 'VC2', 'C3_1.0': 'VC3', 'C4_1.0': 'VC4', 'W1_1.0': 'VW1', 'W2_1.0': 'VW2', 'W3_1.0': 'VW3', 'W4_1.0': 'VW4', 'D1_1.0': 'VD1', 'D2_1.0': 'VD2', 'D3_1.0': 'VD3', 'C1_2.0': 'HC1', 'C2_2.0': 'HC2', 'C3_2.0': 'HC3', 'C4_2.0': 'HC4', 'W1_2.0': 'HW1', 'W2_2.0': 'HW2', 'W3_2.0': 'HW3', 'W4_2.0': 'HW4', 'D1_2.0': 'HD1', 'D2_2.0': 'HD2', 'D3_2.0': 'HD3'})
dy = dy[['Season', 'GameNumber', 'VC1', 'VC2', 'VC3', 'VC4', 'VW1', 'VW2', 'VW3', 'VW4', 'VD1', 'VD2', 'VD3', 'HC1', 'HC2', 'HC3', 'HC4', 'HW1', 'HW2', 'HW3', 'HW4', 'HD1', 'HD2', 'HD3']]
dy.sort_values(['Season', 'GameNumber'], ascending=[True, True], inplace=True)
dy.head()

,Season,GameNumber,VC1,VC2,VC3,VC4,VW1,VW2,VW3,VW4,VD1,VD2,VD3,HC1,HC2,HC3,HC4,HW1,HW2,HW3,HW4,HD1,HD2,HD3
0,2010,20025,1.0,2.0,0.0,1.0,2.0,3.0,1.0,2.0,1.0,2.0,3.0,1.0,0.0,2.0,1.0,2.0,2.0,1.0,3.0,2.0,0.0,4.0
1,2010,20037,0.0,0.0,3.0,1.0,3.0,1.0,0.0,4.0,0.0,3.0,3.0,1.0,1.0,0.0,2.0,2.0,4.0,1.0,1.0,1.0,1.0,4.0
2,2010,20076,1.0,0.0,3.0,0.0,1.0,3.0,2.0,2.0,1.0,1.0,4.0,2.0,0.0,0.0,2.0,2.0,3.0,1.0,2.0,2.0,1.0,3.0
3,2010,20116,1.0,0.0,3.0,0.0,2.0,2.0,1.0,3.0,1.0,0.0,5.0,1.0,0.0,3.0,0.0,1.0,3.0,2.0,2.0,1.0,1.0,4.0
4,2010,20363,1.0,3.0,0.0,0.0,2.0,3.0,2.0,1.0,2.0,3.0,1.0,2.0,0.0,0.0,2.0,3.0,3.0,1.0,1.0,2.0,2.0,2.0


In [189]:
dy.shape

(44, 24)

In [190]:
#dy.to_csv('season_game_roster.csv', index='False', sep=',')

In [191]:
dy.to_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/data/season_game_nhl_positions.csv', index='False', sep=',')
#dy.to_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/data/season_game_nhl_positions.csv', index='False', sep=',')

# Roster Analysis

## season_level_analysis

#### $WinPc = \beta_{0} + \beta_{1}MeanC_{1} + \beta_{2}MeanC_{2}+ \beta_{3}MeanC_{3} + \beta_{4}MeanC_{4} + \beta_{5}MeanW_{1} + \beta_{6}MeanW_{2}+ \beta_{7}MeanW_{3} + \beta_{8}MeanW_{4} + \beta_{9}MeanD_{1} + \beta_{10}MeanD_{2}+ \beta_{11}MeanD_{3} + e_{s}$

#### games with 4 centres, 8 wingers and 6 defensemen

- merge season_team dataset (dz) and season_team_roster_ranking (dv) for roster analysis at the season level. Use **ds** as the merging dataset.

In [192]:
ds = pd.merge(dv, dz, on=['Season', 'TeamCode', 'MeanC', 'MeanW', 'MeanD'], how='left')
ds.sort_values(['MeanC'], ascending=[True], inplace=True)
ds.head(30)

,Season,TeamCode,GP,W,L,GF,GA,WinPc,LossPc,MeanC,MeanW,MeanD,RankWin,RankC,RankW,RankD,GameNumber,RosterCount,GMeanC,GMeanW,GMeanD,C1,C2,C3,C4,D1,D2,D3,W1,W2,W3,W4
4,2010,CAR,9,6,3,30,26,0.666667,0.333333,1.777778,2.361111,1.962963,6.5,1.0,7.0,3.0,20636,18.0,1.543210,1.888889,2.000000,2.0,0.0,2.0,0.0,2.0,2.0,2.0,3.0,2.0,1.0,2.0
12,2010,CGY,7,2,5,22,25,0.285714,0.714286,1.821429,2.267857,1.904762,13.0,2.0,6.0,2.0,20025,18.0,1.777778,1.944444,2.277778,1.0,2.0,0.0,1.0,1.0,2.0,3.0,2.0,3.0,1.0,2.0
16,2010,TOR,1,0,1,1,4,0.000000,1.000000,2.000000,2.750000,2.000000,18.0,3.5,16.0,5.0,20483,18.0,2.388889,2.277778,1.833333,2.0,1.0,0.0,1.0,1.0,4.0,1.0,2.0,2.0,0.0,4.0
13,2010,PHI,4,1,3,12,16,0.250000,0.750000,2.000000,2.218750,1.791667,14.5,3.5,5.0,1.0,20871,18.0,1.902778,2.277778,1.555556,1.0,1.0,2.0,0.0,2.0,4.0,0.0,3.0,2.0,1.0,2.0
9,2010,STL,6,2,4,12,18,0.333333,0.666667,2.125000,2.708333,2.166667,11.0,5.0,15.0,9.0,20493,18.0,2.453704,2.777778,2.111111,2.0,0.0,0.0,2.0,1.0,3.0,2.0,3.0,0.0,1.0,4.0
3,2010,VAN,9,8,1,34,14,0.888889,0.111111,2.416667,2.152778,2.203704,4.0,6.5,4.0,10.0,20076,18.0,2.222222,2.166667,1.944444,2.0,0.0,0.0,2.0,2.0,1.0,3.0,2.0,3.0,1.0,2.0
5,2010,OTT,6,4,2,23,14,0.666667,0.333333,2.416667,3.104167,2.666667,6.5,6.5,18.0,17.5,20929,18.0,2.722222,2.555556,2.833333,1.0,1.0,0.0,2.0,1.0,1.0,4.0,0.0,2.0,4.0,2.0
8,2010,MIN,2,1,1,10,7,0.500000,0.500000,2.500000,2.625000,2.500000,9.0,8.0,12.5,13.5,20076,18.0,2.555556,2.833333,2.500000,1.0,0.0,3.0,0.0,1.0,1.0,4.0,1.0,3.0,2.0,2.0
15,2010,LA,9,2,7,13,25,0.222222,0.777778,2.583333,2.597222,2.240741,16.0,9.0,11.0,11.0,20025,18.0,2.629630,2.722222,2.444444,1.0,0.0,2.0,1.0,2.0,0.0,4.0,2.0,2.0,1.0,3.0
0,2010,ANA,6,6,0,18,9,1.000000,0.000000,2.625000,2.479167,2.027778,2.0,10.0,9.0,7.5,20645,18.0,2.564815,2.611111,2.222222,0.0,1.0,1.0,2.0,2.0,1.0,3.0,3.0,1.0,2.0,2.0


In [193]:
#ds['c1'] = ds.apply(lambda x: 1 if x['MeanC1'] > x['Mean_C'] else 0, 1)
#ds.sort_values(['c1'], ascending=[False], inplace=True)

- A balanced roster will have one player ranked in each centre position (C1, C2, C3, C4), two wingers ranked on every line (LW1, RW1, LW2,RW2, etc) and two defensemen ranked in all three defensive pairings. The ranking of a balanced roster is 2.5 [(1+2+3+4)/4] centres, 2.5 [(1+1+2+2+3+3+4+4)/8] for wingers and 2 [(1+1+2+2+3+3)/6] for defensemen. Since players are ranked from 1 to 4 for forwards and 1 to 3 for defensemen, 1 being the highest ranked. A team is considered to have an above average team when the **mean of each forward position is smaller than 2.5 and the mean of defensive pairings is smaller than 2**. 

In [194]:
ds['C'] = 2.5 - ds['MeanC']
ds['W'] = 2.5 - ds['MeanW']
ds['D'] = 2 - ds['MeanD']

In [195]:
ds.shape

(19, 34)

### summary analysis

In [196]:
ds.describe()

,Season,GP,W,L,GF,GA,WinPc,LossPc,MeanC,MeanW,MeanD,RankWin,RankC,RankW,RankD,GameNumber,RosterCount,GMeanC,GMeanW,GMeanD,C1,C2,C3,C4,D1,D2,D3,W1,W2,W3,W4,C,D
count,19.0,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.0,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.0,19.000000,19.000000,19.000000,19.000000,19.000000
mean,2010.0,4.631579,0.003519,2.315789,12.631579,12.631579,0.477026,0.522974,2.587020,2.496481,2.264237,10.000000,10.000000,10.000000,10.000000,20523.421053,18.0,2.522498,2.362573,2.301170,0.789474,0.684211,1.315789,1.210526,1.157895,1.947368,2.894737,2.0,2.473684,1.210526,2.315789,-0.087020,-0.264237
std,0.0,2.650168,0.359575,1.887168,9.534793,7.544023,0.340239,0.340239,0.458195,0.359575,0.305583,5.570258,5.617433,5.624846,5.587685,375.129950,0.0,0.458278,0.403758,0.365859,0.787327,0.749269,1.416280,0.787327,0.764719,1.223551,1.286457,1.0,1.389181,1.134262,1.249561,0.458195,0.305583
min,2010.0,1.000000,-0.656250,0.000000,1.000000,1.000000,0.000000,0.000000,1.777778,1.750000,1.791667,2.000000,1.000000,1.000000,1.000000,20025.000000,18.0,1.543210,1.611111,1.555556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,-0.666667,-0.750000
25%,2010.0,2.500000,-0.197917,1.000000,5.500000,6.500000,0.250000,0.333333,2.270833,2.243304,2.000000,6.500000,5.750000,5.500000,5.000000,20076.000000,18.0,2.305556,2.138889,2.055556,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,2.000000,1.5,1.500000,0.500000,2.000000,-0.475000,-0.500000
50%,2010.0,4.000000,-0.075000,2.000000,10.000000,12.000000,0.333333,0.666667,2.625000,2.575000,2.203704,11.000000,10.000000,10.000000,10.000000,20602.000000,18.0,2.564815,2.333333,2.277778,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,3.000000,2.0,2.000000,1.000000,2.000000,-0.125000,-0.203704
75%,2010.0,6.000000,0.256696,3.000000,19.500000,17.000000,0.666667,0.750000,2.975000,2.697917,2.500000,14.500000,14.750000,14.500000,13.500000,20863.500000,18.0,2.916667,2.666667,2.500000,1.000000,1.000000,2.500000,2.000000,2.000000,3.000000,4.000000,3.0,3.000000,1.500000,3.000000,0.229167,0.000000
max,2010.0,9.000000,0.750000,7.000000,34.000000,26.000000,1.000000,1.000000,3.166667,3.156250,2.750000,18.000000,19.000000,19.000000,19.000000,21114.000000,18.0,3.148148,3.000000,2.944444,2.000000,2.000000,4.000000,2.000000,2.000000,4.000000,5.000000,3.0,5.000000,4.000000,5.000000,0.722222,0.208333


### estimate roster model 

- regress **team win percent** on the mean of players by position and quality (predictor variables). Add a constant to the predictors and use **OLS**. The purpose is to deterimine the impact each roster position has on team winning percent.

In [197]:
y = ds['WinPc']   
X1 = sm.add_constant(ds[['C', 'W', 'D']] )
m1 = sm.OLS(y, X1).fit()
m1.summary()

/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:1327: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=19
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  WinPc   R-squared:                       0.280
Model:                            OLS   Adj. R-squared:                  0.136
Method:                 Least Squares   F-statistic:                     1.947
Date:                Mon, 12 Mar 2018   Prob (F-statistic):              0.165
Time:                        15:21:06   Log-Likelihood:                -2.8382
No. Observations:                  19   AIC:                             13.68
Df Residuals:                      15   BIC:                             17.45
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          0.2430      0.125      1.947      0.070        -0.023     0.509
C              0.2439      0.225      1.085      0.295        -0.235     0.723
W              0.6187      0.295      2.099      0.053        -0.010     1.247
D             -0.9578      0.429     -2.235      0.041        -1.871    -0.044
==============================================================================
Omnibus:                        3.270   Durbin-Watson:                   1.634
Prob(Omnibus):                  0.195   Jarque-Bera (JB):                1.373
Skew:                           0.448   Prob(JB):                        0.503
Kurtosis:                       3.966   Cond. No.                         7.50
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## season_game_team_level_analysis

#### $Win = \beta_{0} + \beta_{1}MeanC + \beta_{2}MeanW + \beta_{3}MeanD + e_{s,g,t}$

- use season game data (dg) and season game team roster (dx) to conduct season game team level analysis (dt).

In [198]:
dg.head()

,Season,GameNumber,VTeamCode,HTeamCode,VGF,HGF,GD,VGA,HGA,WinTeam,LossTeam
0,2010,20001,MTL,TOR,2,3,1,3,2,TOR,MTL
1,2010,20002,PHI,PIT,3,2,-1,2,3,PHI,PIT
2,2010,20003,CAR,MIN,4,3,-1,3,4,CAR,MIN
3,2010,20004,CHI,COL,3,4,1,4,3,COL,CHI
4,2010,20005,CGY,EDM,0,4,4,4,0,EDM,CGY


In [199]:
dt = dx.merge(dg, on=['Season', 'GameNumber'], how='left')
dt.head()

,Season,GameNumber,TeamCode,RosterCount,MeanC,MeanW,MeanD,GMeanC,GMeanW,GMeanD,C1,C2,C3,C4,D1,D2,D3,W1,W2,W3,W4,VTeamCode,HTeamCode,VGF,HGF,GD,VGA,HGA,WinTeam,LossTeam
0,2010,20025,CGY,18.0,1.821429,2.267857,1.904762,1.777778,1.944444,2.277778,1.0,2.0,0.0,1.0,1.0,2.0,3.0,2.0,3.0,1.0,2.0,LA,CGY,1,3,2,3,1,CGY,LA
1,2010,20025,LA,18.0,2.583333,2.597222,2.240741,2.629630,2.722222,2.444444,1.0,0.0,2.0,1.0,2.0,0.0,4.0,2.0,2.0,1.0,3.0,LA,CGY,1,3,2,3,1,CGY,LA
2,2010,20037,NYI,18.0,2.950000,2.575000,2.500000,2.888889,2.555556,2.500000,0.0,0.0,3.0,1.0,0.0,3.0,3.0,3.0,1.0,0.0,4.0,NYI,WSH,1,2,1,2,1,WSH,NYI
3,2010,20037,WSH,18.0,2.750000,2.125000,2.500000,2.000000,2.333333,2.500000,1.0,1.0,0.0,2.0,1.0,1.0,4.0,2.0,4.0,1.0,1.0,NYI,WSH,1,2,1,2,1,WSH,NYI
4,2010,20076,MIN,18.0,2.500000,2.625000,2.500000,2.555556,2.833333,2.500000,1.0,0.0,3.0,0.0,1.0,1.0,4.0,1.0,3.0,2.0,2.0,VAN,MIN,2,6,4,6,2,MIN,VAN


In [200]:
dt.shape

(88, 30)

- Sum up goals for and against by team per game and find the goal differential (GD) per game. Assign a value of 1 to the team that won the game. 

In [201]:
dt['GD'] = dt.apply(lambda x: (x['HGF'] - x['VGF']) if x['HTeamCode']== x['TeamCode'] else (x['VGF'] - x['HGF']), 1)
dt['Win'] = dt.apply(lambda x: 1 if x['WinTeam']== x['TeamCode'] else 0, 1)
dt['GF'] = dt.apply(lambda x: x['HGF'] if x['HTeamCode']== x['TeamCode'] else x['VGF'], 1)
dt['GA'] = dt.apply(lambda x: x['HGF'] if x['HTeamCode']!= x['TeamCode'] else x['VGF'], 1)
dt.head()

,Season,GameNumber,TeamCode,RosterCount,MeanC,MeanW,MeanD,GMeanC,GMeanW,GMeanD,C1,C2,C3,C4,D1,D2,D3,W1,W2,W3,W4,VTeamCode,HTeamCode,VGF,HGF,GD,VGA,HGA,WinTeam,LossTeam,Win,GF,GA
0,2010,20025,CGY,18.0,1.821429,2.267857,1.904762,1.777778,1.944444,2.277778,1.0,2.0,0.0,1.0,1.0,2.0,3.0,2.0,3.0,1.0,2.0,LA,CGY,1,3,2,3,1,CGY,LA,1,3,1
1,2010,20025,LA,18.0,2.583333,2.597222,2.240741,2.629630,2.722222,2.444444,1.0,0.0,2.0,1.0,2.0,0.0,4.0,2.0,2.0,1.0,3.0,LA,CGY,1,3,-2,3,1,CGY,LA,0,1,3
2,2010,20037,NYI,18.0,2.950000,2.575000,2.500000,2.888889,2.555556,2.500000,0.0,0.0,3.0,1.0,0.0,3.0,3.0,3.0,1.0,0.0,4.0,NYI,WSH,1,2,-1,2,1,WSH,NYI,0,1,2
3,2010,20037,WSH,18.0,2.750000,2.125000,2.500000,2.000000,2.333333,2.500000,1.0,1.0,0.0,2.0,1.0,1.0,4.0,2.0,4.0,1.0,1.0,NYI,WSH,1,2,1,2,1,WSH,NYI,1,2,1
4,2010,20076,MIN,18.0,2.500000,2.625000,2.500000,2.555556,2.833333,2.500000,1.0,0.0,3.0,0.0,1.0,1.0,4.0,1.0,3.0,2.0,2.0,VAN,MIN,2,6,4,6,2,MIN,VAN,1,6,2


In [202]:
dt.shape

(88, 33)

In [204]:
# difference in center position per game
dt['c'] = ((dt['C1']*1) + (dt['C2']*2) + (dt['C3']*3) + (dt['C4']*4)) /4
dt['w'] = ((dt['W1']*1) + (dt['W2']*2) + (dt['W3']*3) + (dt['W4']*4)) /8
dt['d'] = ((dt['D1']*1) + (dt['D2']*2) + (dt['D3']*3)) /6

dt.head()

,Season,GameNumber,TeamCode,RosterCount,MeanC,MeanW,MeanD,GMeanC,GMeanW,GMeanD,C1,C2,C3,C4,D1,D2,D3,W1,W2,W3,W4,VTeamCode,HTeamCode,VGF,HGF,GD,VGA,HGA,WinTeam,LossTeam,Win,GF,GA,c,w,d
0,2010,20025,CGY,18.0,1.821429,2.267857,1.904762,1.777778,1.944444,2.277778,1.0,2.0,0.0,1.0,1.0,2.0,3.0,2.0,3.0,1.0,2.0,LA,CGY,1,3,2,3,1,CGY,LA,1,3,1,2.25,2.375,2.333333
1,2010,20025,LA,18.0,2.583333,2.597222,2.240741,2.629630,2.722222,2.444444,1.0,0.0,2.0,1.0,2.0,0.0,4.0,2.0,2.0,1.0,3.0,LA,CGY,1,3,-2,3,1,CGY,LA,0,1,3,2.75,2.625,2.333333
2,2010,20037,NYI,18.0,2.950000,2.575000,2.500000,2.888889,2.555556,2.500000,0.0,0.0,3.0,1.0,0.0,3.0,3.0,3.0,1.0,0.0,4.0,NYI,WSH,1,2,-1,2,1,WSH,NYI,0,1,2,3.25,2.625,2.500000
3,2010,20037,WSH,18.0,2.750000,2.125000,2.500000,2.000000,2.333333,2.500000,1.0,1.0,0.0,2.0,1.0,1.0,4.0,2.0,4.0,1.0,1.0,NYI,WSH,1,2,1,2,1,WSH,NYI,1,2,1,2.75,2.125,2.500000
4,2010,20076,MIN,18.0,2.500000,2.625000,2.500000,2.555556,2.833333,2.500000,1.0,0.0,3.0,0.0,1.0,1.0,4.0,1.0,3.0,2.0,2.0,VAN,MIN,2,6,4,6,2,MIN,VAN,1,6,2,2.50,2.625,2.500000


### summary analysis

In [251]:
dt.groupby(['Win'])['C1', 'C2', 'C3', 'C4', 'W1', 'W2', 'W3', 'W4', 'D1', 'D2', 'D3'].describe()

C1         C2         C3         C4         W1         W2  \
Win                                                                           
0   count  44.000000  44.000000  44.000000  44.000000  44.000000  44.000000   
    mean    0.863636   1.159091   1.159091   0.818182   2.045455   2.250000   
    std     0.851562   0.938658   1.180130   0.785707   1.033273   1.331637   
    min     0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    25%     0.000000   0.750000   0.000000   0.000000   1.750000   1.000000   
    50%     1.000000   1.000000   1.000000   1.000000   2.000000   2.000000   
    75%     1.250000   2.000000   2.000000   1.000000   3.000000   3.000000   
    max     3.000000   3.000000   4.000000   3.000000   4.000000   5.000000   
1   count  44.000000  44.000000  44.000000  44.000000  44.000000  44.000000   
    mean    1.204545   0.772727   0.909091   1.113636   2.227273   2.318182   
    std     0.733882   0.742830   1.095831   0.841259   1.075383   1.289902   
    min     0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
    25%     1.000000   0.000000   0.000000   0.000000   2.000000   1.000000   
    50%     1.000000   1.000000   1.000000   1.000000   3.000000   2.000000   
    75%     2.000000   1.000000   1.000000   2.000000   3.000000   3.000000   
    max     2.000000   3.000000   4.000000   3.000000   3.000000   5.000000   

                  W3         W4         D1         D2         D3  
Win                                                               
0   count  44.000000  44.000000  44.000000  44.000000  44.000000  
    mean    1.159091   2.545455   1.363636   2.022727   2.613636  
    std     1.119451   1.247407   0.809562   1.171138   1.315664  
    min     0.000000   0.000000   0.000000   0.000000   0.000000  
    25%     0.000000   1.750000   1.000000   1.000000   1.750000  
    50%     1.000000   3.000000   2.000000   2.000000   3.000000  
    75%     2.000000   3.250000   2.000000   3.000000   3.250000  
    max     4.000000   5.000000   2.000000   4.000000   5.000000  
1   count  44.000000  44.000000  44.000000  44.000000  44.000000  
    mean    1.318182   2.136364   1.500000   1.659091   2.840909  
    std     1.115905   1.112109   0.821230   0.775892   1.199674  
    min     0.000000   0.000000   0.000000   0.000000   1.000000  
    25%     1.000000   1.000000   1.000000   1.000000   2.000000  
    50%     1.000000   2.000000   2.000000   2.000000   3.000000  
    75%     2.000000   3.000000   2.000000   2.000000   4.000000  
    max     4.000000   4.000000   3.000000   3.000000   6.000000

In [205]:
dt.groupby(['Win'])['c', 'w', 'd'].describe()

c          w          d
Win                                       
0   count  44.000000  44.000000  44.000000
    mean    2.482955   2.525568   2.208333
    std     0.545589   0.343735   0.307371
    min     1.250000   1.625000   1.666667
    25%     2.000000   2.343750   2.000000
    50%     2.500000   2.500000   2.166667
    75%     3.000000   2.750000   2.500000
    max     3.250000   3.250000   2.833333
1   count  44.000000  44.000000  44.000000
    mean    2.482955   2.420455   2.223485
    std     0.458758   0.389815   0.317335
    min     1.750000   1.625000   1.666667
    25%     2.250000   2.250000   2.000000
    50%     2.500000   2.375000   2.166667
    75%     2.750000   2.625000   2.500000
    max     3.250000   3.250000   3.000000

In [207]:
dt['meanc'] = 2.5 - dt['c']
dt['meanw'] = 2.5 - dt['w']
dt['meand'] = 2 - dt['d']

In [210]:
y = dt['Win']   
X1 = sm.add_constant(dt[['c', 'w', 'd']] )
X2 = sm.add_constant(dt[['meanc', 'meanw', 'meand']] )

m1 = sm.OLS(y, X1).fit()
m2 = sm.OLS(y, X2).fit()

#m1.summary()
m2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    Win   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     1.217
Date:                Mon, 12 Mar 2018   Prob (F-statistic):              0.309
Time:                        16:16:20   Log-Likelihood:                -61.997
No. Observations:                  88   AIC:                             132.0
Df Residuals:                      84   BIC:                             141.9
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          0.4170      0.079      5.288      0.000         0.260     0.574
meanc          0.0659      0.126      0.522      0.603        -0.185     0.317
meanw          0.3480      0.184      1.893      0.062        -0.018     0.714
meand         -0.3357      0.249     -1.347      0.182        -0.831     0.160
==============================================================================
Omnibus:                        0.014   Durbin-Watson:                   2.901
Prob(Omnibus):                  0.993   Jarque-Bera (JB):               12.558
Skew:                           0.028   Prob(JB):                      0.00188
Kurtosis:                       1.150   Cond. No.                         5.64
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [423]:
_var = ['C1', 'C2', 'C3', 'C4', 'W1', 'W2', 'W3', 'W4', 'D1', 'D2', 'D3']
d1 = dt[dt.Win==1][_var].mean()
d2 = dt[dt.Win==0][_var].mean()
df = d1/d2
df

C1    1.043956
C2    0.928571
C3    0.990066
C4    1.060000
W1    1.102249
W2    1.060427
W3    1.080169
W4    0.823009
D1    1.181481
D2    0.984496
D3    0.931774
dtype: float64

### estimate roster model

- regress **win** on the difference in number of players by position and quality per team. Add a constant to the predictors and use **OLS**. The purpose is to deterimine the impact each roster position has on home team success.

In [424]:
dt['mC1'] = dt.C1 - 1
dt['mC2'] = dt.C2 - 1
dt['mC3'] = dt.C3 - 1
dt['mC4'] = dt.C4 - 1

dt['mW1'] = dt.W1 - 2
dt['mW2'] = dt.W2 - 2
dt['mW3'] = dt.W3 - 2
dt['mW4'] = dt.W4 - 2

dt['mD1'] = dt.D1 - 2
dt['mD2'] = dt.D2 - 2
dt['mD3'] = dt.D3 - 2

_m1 = smf.ols('Win ~ mC1 + mC2 + mC3 + mC4', data=dt).fit()
_m2 = smf.ols('Win ~ mW1 + mW2 + mW3 + mW4', data=dt).fit()
_m3 = smf.ols('Win ~ mD1 + mD2 + mD3', data=dt).fit()
_m4 = smf.ols('Win ~ mC1 + mC2 + mC3 +  mW1 + mW2 + mW3 + mW4', data=dt).fit()

_m5 = smf.ols('Win ~ mC1 + mC2 + mC3 + mC4 + mW1 + mW2 + mW3 + mW4 + mD1 + mD2 + mD3', data=dt).fit()
_m6 = smf.logit('Win ~ mC1 + mC2 + mC3 + mC4 +  mW1 + mW2 + mW3 + mW4 + mD1 + mD2 + mD3', data=dt).fit()


summary_col([_m1, _m2, _m3, _m4, _m5, _m6], stars=True)


         Current function value: 0.673577
         Iterations: 35


/anaconda/lib/python3.6/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/anaconda/lib/python3.6/site-packages/statsmodels/iolib/summary2.py:372: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  res.ix[:, 0][idx] = res.ix[:, 0][idx] + '*'
/anaconda/lib/python3.6/site-packages/statsmodels/iolib/summary2.py:374: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  res.ix[:, 0][idx] = res.ix[:, 0][idx] + '*'
/anaconda/lib/python3.6/site-packages/statsmodels/iolib/summary2.py:376: SettingWithCopyWarning: 
A value is trying to be set on a c

,Win I,Win II,Win III,Win IIII,Win IIIII,Win IIIIII
Intercept,0.5041***,0.4969***,0.5491***,0.5231***,0.5669***,0.2803
,(0.0258),(0.0391),(0.0333),(0.0423),(0.0564),(0.2328)
mC1,0.0085,,,-0.0499,-0.0058,-0.0262
,(0.0177),,,(0.0379),(0.0198),(nan)
mC2,-0.0135,,,-0.0531,-0.0200,-0.0848
,(0.0158),,,(0.0333),(0.0172),(nan)
mC3,-0.0074,,,-0.0481,-0.0033,-0.0102
,(0.0219),,,(0.0407),(0.0226),(nan)
mC4,0.0123,,,,0.0290,0.1212
,(0.0238),,,,(0.0256),(nan)


In [425]:
dt['RC1'] = dt.C1 - dt.C4
dt['RC2'] = dt.C2 - dt.C4
dt['RC3'] = dt.C3 - dt.C4

dt['RW1'] = dt.W1 - dt.C4
dt['RW2'] = dt.W2 - dt.C4
dt['RW3'] = dt.W3 - dt.C4
dt['RW4'] = dt.W4 - dt.C4

dt.head()

,Season,GameNumber,TeamCode,RosterCount,MeanC,MeanW,MeanD,GMeanC,GMeanW,GMeanD,C1,C2,C3,C4,D1,D2,D3,W1,W2,W3,W4,VTeamCode,HTeamCode,VGF,HGF,GD,VGA,HGA,WinTeam,LossTeam,Win,GF,GA,c1,c2,c3,c4,w1,w2,w3,w4,d1,d2,d3,mC1,mC2,mC3,mC4,mW1,mW2,mW3,mW4,mD1,mD2,mD3,RC1,RC2,RC3,RW1,RW2,RW3,RW4
0,2010,20005,CGY,18.0,1.923913,2.097826,2.057971,1.855072,2.277778,2.666667,1.0,2.0,0.0,1.0,0.0,3.0,3.0,3.0,2.0,0.0,3.0,CGY,EDM,0,4,-4,4,0,EDM,CGY,0,0,4,0.25,0.50,0.00,0.25,0.375,0.250,0.000,0.375,0.000000,0.500000,0.500000,0.0,1.0,-1.0,0.0,1.0,0.0,-2.0,1.0,-2.0,1.0,1.0,0.0,1.0,-1.0,2.0,1.0,-1.0,2.0
1,2010,20005,EDM,18.0,2.653846,2.605769,2.141026,2.634615,2.000000,2.000000,0.0,2.0,2.0,0.0,0.0,6.0,0.0,1.0,4.0,1.0,2.0,CGY,EDM,0,4,4,4,0,EDM,CGY,1,4,0,0.00,0.50,0.50,0.00,0.125,0.500,0.125,0.250,0.000000,1.000000,0.000000,-1.0,1.0,1.0,-1.0,-1.0,2.0,-1.0,0.0,-2.0,4.0,-2.0,0.0,2.0,2.0,1.0,4.0,1.0,2.0
2,2010,20006,CBJ,18.0,2.140000,2.595000,2.373333,2.191111,2.888889,2.333333,1.0,2.0,1.0,0.0,0.0,3.0,3.0,2.0,2.0,1.0,3.0,SJ,CBJ,3,2,-1,2,3,SJ,CBJ,0,2,3,0.25,0.50,0.25,0.00,0.250,0.250,0.125,0.375,0.000000,0.500000,0.500000,0.0,1.0,0.0,-1.0,0.0,0.0,-1.0,1.0,-2.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,3.0
3,2010,20006,SJ,18.0,1.794118,2.470588,2.137255,1.676471,2.277778,1.833333,3.0,0.0,0.0,1.0,1.0,4.0,1.0,3.0,2.0,0.0,3.0,SJ,CBJ,3,2,1,2,3,SJ,CBJ,1,3,2,0.75,0.00,0.00,0.25,0.375,0.250,0.000,0.375,0.166667,0.666667,0.166667,2.0,-1.0,-1.0,0.0,1.0,0.0,-2.0,1.0,-1.0,2.0,-1.0,2.0,-1.0,-1.0,2.0,1.0,-1.0,2.0
4,2010,20009,ANA,18.0,2.600000,2.737500,1.933333,2.188889,2.388889,2.111111,1.0,1.0,1.0,1.0,2.0,1.0,3.0,3.0,1.0,0.0,4.0,ANA,DET,0,4,-4,4,0,DET,ANA,0,0,4,0.25,0.25,0.25,0.25,0.375,0.125,0.000,0.500,0.333333,0.166667,0.500000,0.0,0.0,0.0,0.0,1.0,-1.0,-2.0,2.0,0.0,-1.0,1.0,0.0,0.0,0.0,2.0,0.0,-1.0,3.0


In [426]:
y = dt['Win']  

X1 = sm.add_constant(dt[['RC1', 'RC2', 'RC3']])


m1 = sm.OLS(y, X1).fit()
m1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    Win   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.003
Method:                 Least Squares   F-statistic:                    0.3751
Date:                Wed, 07 Mar 2018   Prob (F-statistic):              0.771
Time:                        20:42:04   Log-Likelihood:                -407.33
No. Observations:                 562   AIC:                             822.7
Df Residuals:                     558   BIC:                             840.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          0.5041      0.026     19.573      0.000         0.453     0.555
RC1            0.0085      0.018      0.482      0.630        -0.026     0.043
RC2           -0.0135      0.016     -0.852      0.395        -0.045     0.018
RC3           -0.0074      0.022     -0.337      0.736        -0.050     0.036
==============================================================================
Omnibus:                        0.000   Durbin-Watson:                   2.994
Prob(Omnibus):                  1.000   Jarque-Bera (JB):               92.912
Skew:                           0.001   Prob(JB):                     6.67e-21
Kurtosis:                       1.008   Cond. No.                         2.56
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [427]:
y = dt['Win']  
X1 = sm.add_constant(dt[['C1', 'C2', 'C3' , 'W1','W2', 'W3', 'D1', 'D2']] )
X2 = sm.add_constant(dt[['c1', 'c2', 'c3', 'w1', 'w2', 'w3', 'd1', 'd2']] )

#m1 = sm.OLS(y, X1).fit()
m2 = sm.OLS(y, X2).fit()
m2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    Win   R-squared:                       0.038
Model:                            OLS   Adj. R-squared:                  0.024
Method:                 Least Squares   F-statistic:                     2.755
Date:                Wed, 07 Mar 2018   Prob (F-statistic):            0.00544
Time:                        20:42:04   Log-Likelihood:                -396.91
No. Observations:                 562   AIC:                             811.8
Df Residuals:                     553   BIC:                             850.8
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          0.2155      0.162      1.332      0.183        -0.102     0.533
c1            -0.1394      0.158     -0.881      0.379        -0.450     0.171
c2            -0.1960      0.136     -1.445      0.149        -0.462     0.070
c3            -0.1292      0.169     -0.767      0.444        -0.460     0.202
w1             0.6973      0.225      3.106      0.002         0.256     1.138
w2             0.4408      0.164      2.687      0.007         0.119     0.763
w3             0.4018      0.227      1.770      0.077        -0.044     0.848
d1             0.2485      0.213      1.166      0.244        -0.170     0.667
d2            -0.0008      0.150     -0.005      0.996        -0.295     0.293
==============================================================================
Omnibus:                        0.000   Durbin-Watson:                   2.943
Prob(Omnibus):                  1.000   Jarque-Bera (JB):               79.971
Skew:                           0.000   Prob(JB):                     4.31e-18
Kurtosis:                       1.152   Cond. No.                         20.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""